In [ ]:
# !pip install catboost imblearn joblib lightgbm matplotlib numpy optuna mlflow scikit-learn pandas seaborn tqdm ucimlrepo xgboost kaleido

In [ ]:
# Imports padrão do Python
import math
import os
import pickle
import time
import warnings
from collections import Counter, defaultdict
from itertools import combinations
from urllib.parse import urlparse

# Imports de bibliotecas de terceiros
import catboost
import joblib
import lightgbm
import matplotlib.gridspec as GridSpec
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import numpy as np
import optuna
import pandas as pd
import scikit_posthocs as sp
import seaborn as sns
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from optuna.visualization import plot_optimization_history, plot_param_importances
from pandas import DataFrame
from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    roc_auc_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from ucimlrepo import fetch_ucirepo
from xgboost import XGBClassifier
from tqdm import tqdm

In [ ]:
# Configurações
np.random.seed(42)  # Para reprodutibilidade
plt.style.use("ggplot")

# Parâmetros do algoritmo genético
POPULATION_SIZE = 20
MIN_FEATURES = 2
MAX_FEATURES = 14  # Limitado pelo maior k testado
MAX_GENERATIONS = 15
ELITE_PERCENT = 0.2
MUTATION_PROBABILITY = 0.1

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd '/content/drive/MyDrive/TG - Data'

# Data Loading

In [ ]:
national_poll_on_healthy_agig_npha = fetch_ucirepo(id=936)
X = national_poll_on_healthy_agig_npha.data.features
y = national_poll_on_healthy_agig_npha.data.targets
df = pd.merge(X, y, left_index=True, right_index=True)

# 1. EDA

## Checagem básica
Nessa etapa veremos a dimensão, se o dataset possui valores nulos e valores duplicados

In [ ]:
df.info()

In [ ]:
print(f"Qtde Valores Nulos: {df.isnull().sum()}")

print(f"Qtde Valores Duplicados: {df.duplicated().sum()}")

**Mapeamento dos Dados**: Irei alterar a distribuição de alguns dos valores para fazer mais sentido para minha ordenação

In [ ]:
# Mapeamento dos valores das features conforme solicitado
print("Aplicando mapeamento de valores às features...")

# Physical_Health e Mental_Health
health_mapping = {-1: 0, 5: 1, 4: 2, 3: 3, 2: 4, 1: 5}
df["Physical_Health"] = (
    df["Physical_Health"].map(health_mapping).fillna(df["Physical_Health"])
)
df["Mental_Health"] = (
    df["Mental_Health"].map(health_mapping).fillna(df["Mental_Health"])
)

# Dental_Health
dental_mapping = {
    -1: 0,
    5: 1,
    4: 2,
    3: 3,
    2: 4,
    1: 5,
}  # Usando 5 como substituição para 6
# Verificar se algum valor 6 existe e substituir pela mediana se necessário
if 6 in df["Dental_Health"].values:
    median_value = df[df["Dental_Health"].between(1, 5)]["Dental_Health"].median()
    dental_mapping[6] = median_value
df["Dental_Health"] = (
    df["Dental_Health"].map(dental_mapping).fillna(df["Dental_Health"])
)

# Employment
employment_mapping = {
    1: 3,
    2: 2,
    3: 1,
    4: 0,
    -1: 0,
}  # Adicionei -1: 0 para tratar recusas
df["Employment"] = df["Employment"].map(employment_mapping).fillna(df["Employment"])

# Prescription_Sleep_Medication
medication_mapping = {-1: 0, 1: 3, 2: 2, 3: 1}
df["Prescription_Sleep_Medication"] = (
    df["Prescription_Sleep_Medication"]
    .map(medication_mapping)
    .fillna(df["Prescription_Sleep_Medication"])
)

# Mapeamento da target (Number_of_Doctors_Visited)
target_mapping = {1: 0, 2: 1, 3: 2}
print("Aplicando mapeamento de valores à target (Number_of_Doctors_Visited)...")
print(
    f"Distribuição original da target: {df['Number_of_Doctors_Visited'].value_counts()}"
)
df["Number_of_Doctors_Visited"] = (
    df["Number_of_Doctors_Visited"]
    .map(target_mapping)
    .fillna(df["Number_of_Doctors_Visited"])
)
print(f"Distribuição após mapeamento: {df['Number_of_Doctors_Visited'].value_counts()}")

# Converter todas as colunas para inteiros
df = df.astype(int)

print("Mapeamento concluído.")

In [ ]:
feature_mappings = {
    "Number of Doctors Visited": {1: "0-1 doctors", 2: "2-3 doctors", 3: "4+ doctors"},
    "Age": {1: "50-64", 2: "65-80"},
    "Physical Health": {
        0: "Refused",
        1: "Poor",
        2: "Fair",
        3: "Good",
        4: "Very Good",
        5: "Excellent",
    },
    "Mental Health": {
        0: "Refused",
        1: "Poor",
        2: "Fair",
        3: "Good",
        4: "Very Good",
        5: "Excellent",
    },
    "Dental Health": {
        0: "Refused",
        1: "Poor",
        2: "Fair",
        3: "Good",
        4: "Very Good",
        5: "Excellent",
    },
    "Employment": {
        0: "Not working/Refused",
        1: "Retired",
        2: "Working part-time",
        3: "Working full-time",
    },
    "Stress Keeps Patient from Sleeping": {0: "No", 1: "Yes"},
    "Medication Keeps Patient from Sleeping": {0: "No", 1: "Yes"},
    "Pain Keeps Patient from Sleeping": {0: "No", 1: "Yes"},
    "Bathroom Needs Keeps Patient from Sleeping": {0: "No", 1: "Yes"},
    "Uknown Keeps Patient from Sleeping": {0: "No", 1: "Yes"},
    "Trouble Sleeping": {0: "No", 1: "Yes", 2: "Sometimes", 3: "Unknown"},
    "Prescription Sleep Medication": {
        0: "Refused",
        1: "Do not use",
        2: "Use occasionally",
        3: "Use regularly",
    },
    "Race": {
        -2: "Not asked",
        -1: "Refused",
        1: "White, Non-Hispanic",
        2: "Black, Non-Hispanic",
        3: "Other, Non-Hispanic",
        4: "Hispanic",
        5: "2+ Races, Non-Hispanic",
    },
    "Gender": {-2: "Not asked", -1: "Refused", 1: "Male", 2: "Female"},
}

**Conclusão da Etapa**: Nessa etapa vimos que não temos nenhum dado nulo, mas vimos que temos linhas duplicadas no conjunto de dados. Iremos removê-las na etapa de tratamento dos dados pois tratam-se de dados redundantes

## Visualização dos Dados
Nessa etapa iremos utilizar as visualizações de Barplots, Boxplots, Violin Plots e Heatmaps

In [ ]:
# Função para criar um mosaico de barplots
def create_barplot_mosaic(save_image=False, save_path="."):
    # Definir o número de colunas e calcular o número de linhas necessárias
    n_cols = 3
    n_rows = int(np.ceil(len(df.columns) / n_cols))

    # Criar a figura
    plt.figure(figsize=(20, 5 * n_rows))

    # Plotar barplots para cada feature
    for i, col in enumerate(df.columns):
        plt.subplot(n_rows, n_cols, i + 1)

        # Contar os valores para cada categoria
        value_counts = df[col].value_counts().sort_index()

        # Criar barplot
        ax = sns.barplot(x=value_counts.index, y=value_counts.values)

        # Ajustar os ticks do eixo x
        plt.xticks(
            range(len(value_counts)),
            [str(val) for val in value_counts.index],
            rotation=45,
            ha="right",
        )

        plt.title(f"Distribuição de {col}")
        plt.tight_layout()

    plt.suptitle("Mosaico de Barplots para todas as Features", fontsize=20, y=1.02)
    plt.tight_layout()

    if save_image:
        plt.savefig(f"{save_path}/barplot_mosaic.png", dpi=300, bbox_inches="tight")

    plt.show()


# Função para criar um mosaico de boxplots
def create_boxplot_mosaic(save_image=False, save_path="."):
    # Definir o número de colunas e calcular o número de linhas necessárias
    n_cols = 3
    n_rows = int(np.ceil(len(df.columns) / n_cols))

    # Criar a figura
    plt.figure(figsize=(20, 5 * n_rows))

    # Plotar boxplots para cada feature
    for i, col in enumerate(df.columns):
        plt.subplot(n_rows, n_cols, i + 1)

        # Criar boxplot
        sns.boxplot(x=df[col])

        plt.title(f"Boxplot de {col}")
        plt.tight_layout()

    plt.suptitle("Mosaico de Boxplots para todas as Features", fontsize=20, y=1.02)
    plt.tight_layout()

    if save_image:
        plt.savefig(f"{save_path}/boxplot_mosaic.png", dpi=300, bbox_inches="tight")

    plt.show()


# Função para criar um mosaico de violin plots
def create_violinplot_mosaic(save_image=False, save_path="."):
    # Definir o número de colunas e calcular o número de linhas necessárias
    n_cols = 3
    n_rows = int(np.ceil(len(df.columns) / n_cols))

    # Criar a figura
    plt.figure(figsize=(20, 5 * n_rows))

    # Plotar violin plots para cada feature
    for i, col in enumerate(df.columns):
        plt.subplot(n_rows, n_cols, i + 1)

        # Criar violin plot
        sns.violinplot(x=df[col])

        plt.title(f"Violin Plot de {col}")
        plt.tight_layout()

    plt.suptitle("Mosaico de Violin Plots para todas as Features", fontsize=20, y=1.02)
    plt.tight_layout()

    if save_image:
        plt.savefig(f"{save_path}/violinplot_mosaic.png", dpi=300, bbox_inches="tight")

    plt.show()


# Função para criar um mapa de calor de correlação
def create_correlation_heatmap(save_image=False, save_path=".", linear_corr=True):
    # Criar a figura
    plt.figure(figsize=(16, 14))

    # Calcular a matriz de correlação
    if linear_corr:
        corr_matrix = df.corr(method="pearson")
    else:
        corr_matrix = df.corr(method="spearman")

    # Criar o mapa de calor
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(
        corr_matrix,
        annot=True,
        fmt=".2f",
        cmap="coolwarm",
        mask=mask,
        square=True,
        linewidths=0.5,
    )

    plt.title("Mapa de Calor de Correlação", fontsize=20)
    plt.tight_layout()

    if save_image:
        plt.savefig(
            f"{save_path}/correlation_heatmap.png", dpi=300, bbox_inches="tight"
        )

    plt.show()


# Função para criar um mosaico de scatter plots entre o target e as features
def create_scatter_mosaic(save_image=False, save_path="."):
    target = "Number_of_Doctors_Visited"
    features = [col for col in df.columns if col != target]

    # Definir o número de colunas e calcular o número de linhas necessárias
    n_cols = 3
    n_rows = int(np.ceil(len(features) / n_cols))

    # Criar a figura
    plt.figure(figsize=(20, 5 * n_rows))

    # Mapear as cores para os valores do target
    colors = {1: "blue", 2: "green", 3: "red"}

    # Plotar a relação de cada feature com o target usando scatter plot
    for i, feature in enumerate(features):
        plt.subplot(n_rows, n_cols, i + 1)

        # Adicionar jitter para melhor visualização (dados discretos podem se sobrepor)
        x = df[feature] + np.random.normal(0, 0.1, size=len(df))
        y = df[target] + np.random.normal(0, 0.1, size=len(df))

        # Criar scatter plot com cores baseadas no target
        for target_val in sorted(df[target].unique()):
            mask = df[target] == target_val
            plt.scatter(
                x[mask],
                y[mask],
                alpha=0.5,
                label=f"Classe {target_val}",
                color=colors.get(target_val, f"C{target_val}"),
            )

        # Ajustar limites para melhor visualização
        plt.xlim(df[feature].min() - 0.5, df[feature].max() + 0.5)
        plt.ylim(df[target].min() - 0.5, df[target].max() + 0.5)

        # Ajustar ticks para mostrar apenas os valores originais
        plt.xticks(sorted(df[feature].unique()))
        plt.yticks(sorted(df[target].unique()))

        # Adicionar linhas de grade para facilitar a visualização
        plt.grid(True, linestyle="--", alpha=0.7)

        plt.xlabel(feature)
        plt.ylabel(target)
        plt.title(f"Relação entre {feature} e {target}")

        # Adicionar legenda apenas no primeiro gráfico
        if i == 0:
            plt.legend()

    plt.suptitle(
        f"Scatter Plots: Relações entre Features e {target}", fontsize=20, y=1.02
    )
    plt.tight_layout()

    if save_image:
        plt.savefig(f"{save_path}/scatter_mosaic.png", dpi=300, bbox_inches="tight")

    plt.show()

In [ ]:
create_barplot_mosaic()

In [ ]:
create_boxplot_mosaic()

In [ ]:
create_violinplot_mosaic()

In [ ]:
create_scatter_mosaic()

In [ ]:
create_correlation_heatmap()

**Conclusão**: Ao analisar as features, tivemos as seguintes conclusões
1. No geral, muitas features do dataset apresentam pouca variabilidade
2. A target não tem uma correlaÇão linear muito forte com nenhuma feature
3. O número de visitas aos médicos não tem correlação com o estado mental dos indivíduos. Isso é inesperado.
4. A coluna de idade e gênero não tem correlação nenhuma com a target e podem ser dropada.
5. Por termos apenas variáveis categóricas e binárias, não teremos necessidade
de normalizar as features
6. O ideal é que usemos o SMOTE na etapa de treinamento devido ao desbalancemanto das features
7. Não temos colunas duplas, então não precisaremos fazer imputação das features, mas precisaremos remover as features duplicadas.
8. Um repameanto dos valores das features foi feito para fazer mais sentido e
precisamos mudar o valor '6' de Dental_Health para o valor da mediana, já que ele não estava presente
na descrição do dataset, indicando um ruído.
9. Trouble sleeping terá que ser dropada pois ela deveria ser binária e apresenta 3 valores (1, 2 e 3). Isso indica forte ruído e não tenho ideias
de como corrigir isso

# 2. Data Cleaning
Visando a Preparar o Conjunto de Dados para a modelagem, nessa etapa:
1. Iremos remover as linhas duplicadas
2. Iremos dividir os dados em 10 conjuntos menores utilizando o StratifiedKfold, além de enriquecermos os dados de treino utilizando SMOTE


In [ ]:
df = df.drop_duplicates()

In [ ]:
# Preparar X e y
X = df.drop("Number_of_Doctors_Visited", axis=1)
y = df["Number_of_Doctors_Visited"]

# Inicializar StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True)

# Inicializar SMOTE
smote = SMOTE()

# Lista para armazenar os folds
smote_folds_data = []
original_folds_data = []

# Para cada fold
for train_index, test_index in skf.split(X, y):
    # Obter os subconjuntos originais
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Aplicar SMOTE no conjunto de treino
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    # Converter para DataFrame se X_train_smote retornar um array
    if not isinstance(X_train_smote, pd.DataFrame):
        X_train_smote = pd.DataFrame(X_train_smote, columns=X.columns)

    # Mostrar distribuição das classes antes e depois do SMOTE
    print(f"Fold - Distribuição original: {Counter(y_train)}")
    print(f"Fold - Distribuição após SMOTE: {Counter(y_train_smote)}")

    # Adicionar o fold à lista
    smote_folds_data.append((X_train_smote, X_test, y_train_smote, y_test))
    original_folds_data.append((X_train, X_test, y_train, y_test))

# 3. Feature Engineering
Nesta etapa iremos escolher as melhores features do dataset com a seguinte metodologia:
1. Utilizaremos LR, RF, KNN, NB e Catboost para avaliar os datasets.
2. Separaremos uma cópia do dataset original
3. Utilizaremos o método anova e selecionaremos *k* features, com o valor de *k* variando entre 7 e 11 features, em que tiraremos a media do AUC_SCORE para o grupo de classificadores avaliadores. O *k* com melhor média será escolhido para a avalição final.
4. Utilizaremos algoritmo genético junto com random forests para selecionar o melhor subconjunto de features utilizando uma abordagem do tipo *wrapper*
5. Selecionaremos as features com melhor média de auc_score para o conjunto de classificadores.

## Funções MlFlow Tracking

In [ ]:
def setup_mlflow(experiment_name="Busca das Features", sqlite_path="mlruns.db"):
    """
    Configura o MLflow para registrar experimentos em um banco de dados SQLite

    Args:
        experiment_name: Nome do experimento
        sqlite_path: Caminho para o arquivo SQLite

    Returns:
        experiment_id: ID do experimento criado ou recuperado
    """
    # Configurar o tracking URI para o SQLite
    mlflow_tracking_uri = f"sqlite:///{sqlite_path}"
    mlflow.set_tracking_uri(mlflow_tracking_uri)

    # Criar ou obter o experimento
    try:
        experiment_id = mlflow.create_experiment(
            experiment_name, artifact_location=os.path.join(os.getcwd(), "mlruns")
        )
        print(f"Experimento '{experiment_name}' criado com ID: {experiment_id}")
    except mlflow.exceptions.MlflowException:
        # Experimento já existe, recuperar ID
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
        print(f"Experimento '{experiment_name}' já existe com ID: {experiment_id}")

    mlflow.set_experiment(experiment_name)

    return experiment_id


def log_feature_selection_results(
    model_name, model, method, features, mean_auc, std_auc, fold_aucs, k=None
):
    """
    Registra os resultados de um método de seleção de features no MLflow

    Args:
        model_name: Nome do modelo
        model: Objeto do modelo scikit-learn
        method: Nome do método de seleção de features ('all', 'anova', 'ga')
        features: Lista de features selecionadas
        mean_auc: AUC-ROC médio
        std_auc: Desvio padrão do AUC-ROC
        fold_aucs: Lista de AUC-ROC para cada fold
        k: Valor de k para o método ANOVA (opcional)
    """
    # Mapear o nome do método para um nome mais descritivo
    method_names = {
        "all": "Todas as Features",
        "anova": f"ANOVA (k={k})" if k else "ANOVA",
        "ga": "Algoritmo Genético",
    }

    # Iniciar o run com o nome do modelo e método
    run_name = f"{model_name} - {method_names[method]}"

    with mlflow.start_run(run_name=run_name):
        # Registrar os parâmetros
        params = {
            "model_name": model_name,
            "feature_selection_method": method_names[method],
            "num_features": len(features),
        }

        # Adicionar k se o método for ANOVA
        if method == "anova" and k is not None:
            params["anova_k"] = k

        # Registrar os parâmetros do modelo se disponíveis
        if hasattr(model, "get_params"):
            model_params = model.get_params()
            for param_name, param_value in model_params.items():
                # Converter qualquer valor não serializado para string
                if not isinstance(param_value, (int, float, str, bool, type(None))):
                    param_value = str(param_value)
                params[f"model_{param_name}"] = param_value

        # Registrar todos os parâmetros
        mlflow.log_params(params)

        # Registrar as métricas
        metrics = {
            "mean_auc": mean_auc,
            "std_auc": std_auc,
            "min_auc": np.min(fold_aucs),
            "max_auc": np.max(fold_aucs),
            "median_auc": np.median(fold_aucs),
        }

        # Registrar cada fold individualmente
        for i, fold_auc in enumerate(fold_aucs):
            metrics[f"fold_{i+1}_auc"] = fold_auc

        # Registrar todas as métricas
        mlflow.log_metrics(metrics)

        # Registrar as features como um artefato
        pd.DataFrame({"feature": features}).to_csv("selected_features.csv", index=False)
        mlflow.log_artifact("selected_features.csv")

        # Registrar o modelo se aplicável
        try:
            mlflow.sklearn.log_model(model, "model")
        except Exception as e:
            print(f"Aviso: Não foi possível registrar o modelo: {e}")

        # Registrar link para o tracking server
        tracking_url = mlflow.get_tracking_uri()
        print(f"MLflow Tracking URL: {tracking_url}")

        # Capturar o ID do run para referência
        run_id = mlflow.active_run().info.run_id
        print(f"Run ID: {run_id}")

        return run_id


def log_all_results_to_mlflow(results):
    """
    Registra todos os resultados de seleção de features no MLflow

    Args:
        results: Dicionário com resultados por modelo e método de seleção
    """
    # Configurar MLflow
    experiment_id = setup_mlflow()

    # Obter dicionário de modelos

    model_dict = get_model_dict()

    # Para cada modelo
    for model_name, model_results in results.items():
        print(f"\nRegistrando resultados para {model_name}...")

        # Obter o objeto do modelo
        model = model_dict.get(model_name)

        # Registrar resultados para todas as features
        all_features_results = model_results["all_features"]
        log_feature_selection_results(
            model_name=model_name,
            model=model,
            method="all",
            features=all_features_results["features"],
            mean_auc=all_features_results["mean_auc"],
            std_auc=all_features_results["std_auc"],
            fold_aucs=all_features_results["fold_aucs"],
        )

        # Registrar resultados para ANOVA
        best_k = model_results["anova"]["best_k"]
        anova_results = model_results["anova"]["best_results"]
        log_feature_selection_results(
            model_name=model_name,
            model=model,
            method="anova",
            features=anova_results["features"],
            mean_auc=anova_results["mean_auc"],
            std_auc=anova_results["std_auc"],
            fold_aucs=anova_results["fold_aucs"],
            k=best_k,
        )

        # Registrar resultados para Algoritmo Genético
        ga_results = model_results["ga"]
        fold_aucs = [result["auc"] for result in ga_results["fold_results"]]
        log_feature_selection_results(
            model_name=model_name,
            model=model,
            method="ga",
            features=ga_results["common_features"],
            mean_auc=ga_results["mean_auc"],
            std_auc=ga_results["std_auc"],
            fold_aucs=fold_aucs,
        )

    print(
        f"\nTodos os resultados foram registrados no experimento MLflow (ID: {experiment_id})"
    )
    print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")

    # Instruções para visualizar os resultados
    print(
        "\nPara visualizar os resultados no UI do MLflow, execute o seguinte comando no terminal:"
    )
    print(f"mlflow ui --backend-store-uri {mlflow.get_tracking_uri()}")


def log_best_models_to_mlflow(results):
    """
    Registra apenas os melhores modelos/métodos no MLflow

    Args:
        results: Dicionário com resultados por modelo e método de seleção
    """
    # Configurar MLflow
    experiment_name = "Melhores Modelos de Seleção de Features"
    experiment_id = setup_mlflow(experiment_name)

    model_dict = get_model_dict()

    best_models = []

    # Para cada modelo
    for model_name, model_results in results.items():
        # Obter o objeto do modelo
        model = model_dict.get(model_name)

        # Obter AUC para cada método
        all_features_auc = model_results["all_features"]["mean_auc"]
        anova_auc = model_results["anova"]["best_results"]["mean_auc"]
        ga_auc = model_results["ga"]["mean_auc"]

        # Determinar o melhor método
        methods = ["all", "anova", "ga"]
        method_names = ["Todas as Features", "ANOVA", "Algoritmo Genético"]
        aucs = [all_features_auc, anova_auc, ga_auc]
        best_idx = np.argmax(aucs)
        best_method = methods[best_idx]
        best_method_name = method_names[best_idx]

        # Obter dados do melhor método
        if best_method == "all":
            best_results = model_results["all_features"]
            features = best_results["features"]
            fold_aucs = best_results["fold_aucs"]
            k = None
        elif best_method == "anova":
            best_k = model_results["anova"]["best_k"]
            best_results = model_results["anova"]["best_results"]
            features = best_results["features"]
            fold_aucs = best_results["fold_aucs"]
            k = best_k
        else:  # ga
            best_results = model_results["ga"]
            features = best_results["common_features"]
            fold_aucs = [result["auc"] for result in best_results["fold_results"]]
            k = None

        # Registrar o melhor método
        run_id = log_feature_selection_results(
            model_name=model_name,
            model=model,
            method=best_method,
            features=features,
            mean_auc=best_results["mean_auc"],
            std_auc=best_results["std_auc"],
            fold_aucs=fold_aucs,
            k=k,
        )

        # Adicionar à lista de melhores modelos
        best_models.append(
            {
                "model_name": model_name,
                "best_method": best_method_name,
                "auc": best_results["mean_auc"],
                "num_features": len(features),
                "run_id": run_id,
            }
        )

    # Criar DataFrame para resumo dos melhores modelos
    best_df = pd.DataFrame(best_models)
    best_df = best_df.sort_values("auc", ascending=False).reset_index(drop=True)

    # Salvar e registrar como artefato
    with mlflow.start_run(run_name="Resumo dos Melhores Modelos"):
        best_df.to_csv("best_models_summary.csv", index=False)
        mlflow.log_artifact("best_models_summary.csv")

        # Registrar métricas gerais
        mlflow.log_metric("num_models", len(best_df))
        mlflow.log_metric("max_auc", best_df["auc"].max())
        mlflow.log_metric("mean_auc", best_df["auc"].mean())

        # Contar quantas vezes cada método foi o melhor
        method_counts = best_df["best_method"].value_counts()
        for method, count in method_counts.items():
            mlflow.log_metric(f"{method.replace(' ', '_')}_count", count)

    print(
        f"\nResumo dos melhores modelos registrado no experimento MLflow (ID: {experiment_id})"
    )
    print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")

## Funções Auxiliares

In [ ]:
def get_model_dict():
    return {
        "Decision Trees": DecisionTreeClassifier(random_state=42),
        "Gradient Tree Boosting": GradientBoostingClassifier(random_state=42),
        "k-Nearest Neighbors": KNeighborsClassifier(),
        "LightGBM": LGBMClassifier(random_state=42),
        "Multinomial Logistic Regression": LogisticRegression(
            multi_class="multinomial", solver="lbfgs", max_iter=1000, random_state=42
        ),
        "Naive Bayes": GaussianNB(),
        "Random Forests": RandomForestClassifier(random_state=42),
        "Support Vector Machines": SVC(probability=True, random_state=42),
        "XGBoost": XGBClassifier(random_state=42),
        "CatBoost": CatBoostClassifier(verbose=False, random_state=42),
    }


# 1. Seleção de features por ANOVA para cada modelo
def select_features_anova(X_train, y_train, k):
    """Seleciona k features usando o método ANOVA"""
    selector = SelectKBest(f_classif, k=k)
    selector.fit(X_train, y_train)

    # Obter os índices das features selecionadas
    selected_indices = selector.get_support(indices=True)

    # Obter os nomes das features selecionadas
    selected_features = X_train.columns[selected_indices]

    return selected_features


# 2. Avaliar conjunto de features com um classificador específico
def evaluate_features_with_model(model, X_train, X_test, y_train, y_test, features):
    """Avalia um conjunto de features usando um classificador específico"""
    # Selecionar apenas as features escolhidas
    X_train_selected = X_train[features]
    X_test_selected = X_test[features]

    # Clonar o modelo para evitar contaminação
    model_clone = clone_model(model)

    # Verificar se as classes já começam em 0 (apenas para logging)
    min_class = min(np.unique(y_train))
    if min_class == 0:
        print(f"Classes já começam em 0: {sorted(np.unique(y_train))}")

    # Treinar o modelo diretamente (sem mapeamento de classes)
    try:
        # Treinar o modelo
        model_clone.fit(X_train_selected, y_train)

        # Calcular AUC score
        if hasattr(model_clone, "predict_proba"):
            y_prob = model_clone.predict_proba(X_test_selected)
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
        else:
            # Para classificadores que não têm predict_proba
            auc = 0.5  # Valor padrão, pior cenário
    except Exception as e:
        print(f"Erro ao treinar/avaliar modelo {type(model_clone).__name__}: {e}")
        auc = 0.5  # Valor padrão em caso de erro

    return auc


# Função auxiliar para clonar um modelo
def clone_model(model):
    """Cria uma cópia nova do modelo com os mesmos parâmetros"""
    from sklearn.base import clone

    return clone(model)


# 3. Testar diferentes valores de k com ANOVA para um modelo específico
def evaluate_anova_for_model(model, folds_data, k_values=range(7, 15)):
    """Testa ANOVA com diferentes valores de k para um modelo específico em todos os folds"""
    k_results = {}
    best_k = None
    best_mean_auc = 0

    for k in k_values:
        fold_aucs = []

        for fold_idx, (X_train, X_test, y_train, y_test) in enumerate(folds_data):
            # Selecionar features usando ANOVA
            selected_features = select_features_anova(X_train, y_train, k)

            # Avaliar com o modelo específico
            auc = evaluate_features_with_model(
                model, X_train, X_test, y_train, y_test, selected_features
            )
            fold_aucs.append(auc)

        # Calcular média e desvio padrão dos AUCs nos folds
        mean_auc = np.mean(fold_aucs)
        std_auc = np.std(fold_aucs)

        # Armazenar resultados
        k_results[k] = {
            "fold_aucs": fold_aucs,
            "mean_auc": mean_auc,
            "std_auc": std_auc,
            "features": selected_features,  # Usa o último fold como referência para features
        }

        # Verificar se este é o melhor k
        if mean_auc > best_mean_auc:
            best_mean_auc = mean_auc
            best_k = k

    return k_results, best_k

## Funções do Algoritmo Genético


In [ ]:
# 4. Funções para o algoritmo genético
def generate_initial_population(n_features):
    """Gera a população inicial de cromossomos"""
    population = np.zeros((POPULATION_SIZE, n_features))

    for i in range(POPULATION_SIZE):
        # Determinar quantas features selecionar (entre MIN_FEATURES e MAX_FEATURES)
        n_selected = np.random.randint(MIN_FEATURES, min(MAX_FEATURES + 1, n_features))

        # Escolher aleatoriamente quais features selecionar
        selected_indices = np.random.choice(n_features, n_selected, replace=False)
        population[i, selected_indices] = 1

    return population


def evaluate_chromosome(
    model, X_train, X_test, y_train, y_test, chromosome, feature_names
):
    """Avalia um cromossomo usando o modelo especificado"""
    # Selecionar as features de acordo com o cromossomo
    selected_features = feature_names[chromosome == 1]

    if len(selected_features) == 0:
        # Se nenhuma feature for selecionada, atribuir AUC zero
        return 0

    # Avaliar usando o modelo
    auc = evaluate_features_with_model(
        model, X_train, X_test, y_train, y_test, selected_features
    )

    return auc


def evaluate_population(
    model, population, X_train, X_test, y_train, y_test, feature_names
):
    """Avalia toda a população de cromossomos"""
    fitness_scores = np.zeros(POPULATION_SIZE)

    for i in range(POPULATION_SIZE):
        fitness_scores[i] = evaluate_chromosome(
            model, X_train, X_test, y_train, y_test, population[i], feature_names
        )

    return fitness_scores


def select_parents(population, fitness_scores):
    """Seleciona pais usando Roulette Wheel Selection"""
    # Número de elite (melhores indivíduos que serão mantidos para a próxima geração)
    elite_count = int(POPULATION_SIZE * ELITE_PERCENT)

    # Índices dos elite_count melhores indivíduos
    elite_indices = np.argsort(fitness_scores)[-elite_count:]

    # Seleção dos pais para o restante da população usando roleta
    weights = fitness_scores / (
        np.sum(fitness_scores) + 1e-10
    )  # Normalizar para soma = 1
    cumulative_weights = np.cumsum(weights)

    parents = np.zeros_like(population)

    # Primeiro adicionar a elite
    parents[:elite_count] = population[elite_indices]

    # Depois selecionar o restante usando a roleta
    for i in range(elite_count, POPULATION_SIZE):
        r = np.random.random()
        idx = np.searchsorted(cumulative_weights, r)
        if idx >= len(population):
            idx = len(population) - 1
        parents[i] = population[idx]

    return parents


def crossover_and_mutate(parents, n_features):
    """Aplica crossover e mutação nos pais para gerar filhos"""
    # Número de elite que será preservado
    elite_count = int(POPULATION_SIZE * ELITE_PERCENT)

    # Criar array para a nova população
    children = np.zeros_like(parents)

    # Manter a elite
    children[:elite_count] = parents[:elite_count]

    # Aplicar crossover no restante
    for i in range(elite_count, POPULATION_SIZE, 2):
        if i + 1 < POPULATION_SIZE:
            # Selecionar dois pais aleatoriamente
            parent_idx1 = np.random.randint(0, POPULATION_SIZE)
            parent_idx2 = np.random.randint(0, POPULATION_SIZE)

            # Escolher ponto de crossover
            crossover_point = np.random.randint(1, n_features)

            # Criar filhos com o crossover
            children[i, :crossover_point] = parents[parent_idx1, :crossover_point]
            children[i, crossover_point:] = parents[parent_idx2, crossover_point:]

            children[i + 1, :crossover_point] = parents[parent_idx2, :crossover_point]
            children[i + 1, crossover_point:] = parents[parent_idx1, crossover_point:]

    # Aplicar mutação em cada filho, exceto na elite
    for i in range(elite_count, POPULATION_SIZE):
        for j in range(n_features):
            if np.random.random() < MUTATION_PROBABILITY:
                # Inverter o bit (0->1 ou 1->0)
                children[i, j] = 1 - children[i, j]

        # Garantir que o número de features esteja dentro dos limites
        selected_count = np.sum(children[i])

        if selected_count < MIN_FEATURES:
            # Adicionar features se abaixo do mínimo
            zeros_indices = np.where(children[i] == 0)[0]
            if len(zeros_indices) > 0:
                add_count = min(MIN_FEATURES - selected_count, len(zeros_indices))
                add_indices = np.random.choice(
                    zeros_indices, int(add_count), replace=False
                )
                children[i, add_indices] = 1

        elif selected_count > MAX_FEATURES:
            # Remover features se acima do máximo
            ones_indices = np.where(children[i] == 1)[0]
            if len(ones_indices) > 0:
                remove_count = min(selected_count - MAX_FEATURES, len(ones_indices))
                remove_indices = np.random.choice(
                    ones_indices, int(remove_count), replace=False
                )
                children[i, remove_indices] = 0

    return children


def run_genetic_algorithm_for_model(model, X_train, X_test, y_train, y_test):
    """Executa o algoritmo genético para um modelo específico em um fold específico"""
    n_features = X_train.shape[1]
    feature_names = X_train.columns

    # Inicializar população
    population = generate_initial_population(n_features)

    # Armazenar o melhor fitness de cada geração
    best_fitness_history = np.zeros(MAX_GENERATIONS)
    best_chromosome = None
    best_fitness = 0

    # Executar por MAX_GENERATIONS gerações
    for generation in range(MAX_GENERATIONS):
        # Avaliar população atual
        fitness_scores = evaluate_population(
            model, population, X_train, X_test, y_train, y_test, feature_names
        )

        # Armazenar o melhor dessa geração
        generation_best_idx = np.argmax(fitness_scores)
        generation_best_fitness = fitness_scores[generation_best_idx]
        best_fitness_history[generation] = generation_best_fitness

        # Atualizar o melhor global
        if generation_best_fitness > best_fitness:
            best_fitness = generation_best_fitness
            best_chromosome = population[generation_best_idx].copy()

        # Selecionar pais
        parents = select_parents(population, fitness_scores)

        # Criar nova população com crossover e mutação
        population = crossover_and_mutate(parents, n_features)

    # Selecionar as features com base no melhor cromossomo
    if best_chromosome is not None:
        selected_features = feature_names[best_chromosome == 1]
    else:
        # Caso de falha, selecionar algumas features aleatórias
        selected_count = min(MAX_FEATURES, len(feature_names))
        selected_features = np.random.choice(
            feature_names, selected_count, replace=False
        )

    return selected_features, best_fitness, best_fitness_history


def evaluate_ga_for_model(model, folds_data):
    """Avalia o algoritmo genético para um modelo específico em todos os folds"""
    fold_results = []
    all_selected_features = []

    for fold_idx, (X_train, X_test, y_train, y_test) in enumerate(folds_data):
        # Executar GA para este fold
        selected_features, best_fitness, _ = run_genetic_algorithm_for_model(
            model, X_train, X_test, y_train, y_test
        )

        fold_results.append(
            {"fold_idx": fold_idx, "auc": best_fitness, "features": selected_features}
        )

        all_selected_features.append(selected_features)

    # Calcular média e desvio padrão de AUC
    aucs = [result["auc"] for result in fold_results]
    mean_auc = np.mean(aucs)
    std_auc = np.std(aucs)

    # Encontrar as features mais comuns entre todos os folds
    # Vamos contar quantas vezes cada feature aparece
    feature_counts = {}
    for features in all_selected_features:
        for feature in features:
            if feature in feature_counts:
                feature_counts[feature] += 1
            else:
                feature_counts[feature] = 1

    # Ordenar features por frequência
    sorted_features = sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)

    # Pegar as top MAX_FEATURES mais frequentes
    common_features = [feature for feature, count in sorted_features[:MAX_FEATURES]]

    return {
        "fold_results": fold_results,
        "mean_auc": mean_auc,
        "std_auc": std_auc,
        "common_features": common_features,
    }

## Funções de Seleção das Melhores Features

In [ ]:
# 5. Avaliar todas as features para um modelo específico
def evaluate_all_features_for_model(model, folds_data):
    """Avalia todas as features para um modelo específico em todos os folds"""
    fold_aucs = []

    for fold_idx, (X_train, X_test, y_train, y_test) in enumerate(folds_data):
        # Usar todas as features
        all_features = X_train.columns

        # Avaliar com o modelo específico
        auc = evaluate_features_with_model(
            model, X_train, X_test, y_train, y_test, all_features
        )
        fold_aucs.append(auc)

    # Calcular média e desvio padrão dos AUCs nos folds
    mean_auc = np.mean(fold_aucs)
    std_auc = np.std(fold_aucs)

    return {
        "fold_aucs": fold_aucs,
        "mean_auc": mean_auc,
        "std_auc": std_auc,
        "features": all_features,
    }

## Avaliação das Features

In [ ]:
def evaluate_all_models_with_all_feature_selection_methods(folds_data):
    """
    Avalia todos os modelos usando diferentes métodos de seleção de features

    Args:
        folds_data: Lista de tuplas (X_train, X_test, y_train, y_test) para cada fold

    Returns:
        dict: Resultados por modelo e método de seleção de features
    """
    # Obter a lista de modelos
    model_dict = get_model_dict()

    # Dicionário para armazenar resultados
    results = {}

    # Valores de k para ANOVA
    k_values = range(7, 15)  # 7 a 14 features

    # Avaliar cada modelo
    for model_name, model in tqdm(model_dict.items(), desc="Avaliando modelos"):
        print(f"\n===== Avaliando {model_name} =====")
        model_results = {}

        # 1. Avaliar com todas as features
        print("Avaliando com todas as features...")
        all_features_results = evaluate_all_features_for_model(model, folds_data)
        model_results["all_features"] = all_features_results
        print(
            f"  AUC médio: {all_features_results['mean_auc']:.4f} ± {all_features_results['std_auc']:.4f}"
        )

        # 2. Avaliar com ANOVA
        print("Avaliando com seleção ANOVA...")
        anova_results, best_k = evaluate_anova_for_model(model, folds_data, k_values)
        model_results["anova"] = {
            "all_k_results": anova_results,
            "best_k": best_k,
            "best_results": anova_results[best_k],
        }
        print(f"  Melhor k: {best_k}")
        print(
            f"  AUC médio: {anova_results[best_k]['mean_auc']:.4f} ± {anova_results[best_k]['std_auc']:.4f}"
        )
        print(f"  Features: {', '.join(anova_results[best_k]['features'])}")

        # 3. Avaliar com Algoritmo Genético
        print("Avaliando com Algoritmo Genético...")
        ga_results = evaluate_ga_for_model(model, folds_data)
        model_results["ga"] = ga_results
        print(
            f"  AUC médio: {ga_results['mean_auc']:.4f} ± {ga_results['std_auc']:.4f}"
        )
        print(f"  Features comuns: {', '.join(ga_results['common_features'])}")

        # Armazenar resultados do modelo
        results[model_name] = model_results

        # Salvar resultados parciais (após cada modelo)
        save_results(results, "feature_selection_results.pkl")

    return results


def save_results(results, filename):
    """Salva os resultados em um arquivo pickle"""
    with open(filename, "wb") as f:
        pickle.dump(results, f)
    print(f"Resultados salvos em {filename}")


def load_results(filename):
    """Carrega os resultados de um arquivo pickle"""
    with open(filename, "rb") as f:
        results = pickle.load(f)
    return results


def save_best_features_to_txt(results, filename="best_features.txt"):
    """
    Salva as melhores features e AUC-ROC para cada algoritmo em um arquivo texto

    Args:
        results: Dicionário com resultados por modelo e método de seleção
        filename: Nome do arquivo de saída
    """
    with open(filename, "w") as f:
        f.write("MELHORES FEATURES E AUC-ROC POR ALGORITMO\n")
        f.write("=" * 80 + "\n\n")

        for model_name, model_results in results.items():
            f.write(f"Modelo: {model_name}\n")
            f.write("-" * 80 + "\n")

            # Todas as features
            all_features_auc = model_results["all_features"]["mean_auc"]
            all_features_std = model_results["all_features"]["std_auc"]
            all_features_list = model_results["all_features"]["features"]

            f.write(f"Todas as Features ({len(all_features_list)}):\n")
            f.write(f"  AUC-ROC: {all_features_auc:.4f} ± {all_features_std:.4f}\n")
            f.write(f"  Features: {', '.join(all_features_list)}\n\n")

            # ANOVA
            best_k = model_results["anova"]["best_k"]
            anova_auc = model_results["anova"]["best_results"]["mean_auc"]
            anova_std = model_results["anova"]["best_results"]["std_auc"]
            anova_features = model_results["anova"]["best_results"]["features"]

            f.write(f"ANOVA (k={best_k}):\n")
            f.write(f"  AUC-ROC: {anova_auc:.4f} ± {anova_std:.4f}\n")
            f.write(f"  Features: {', '.join(anova_features)}\n\n")

            # Algoritmo Genético
            ga_auc = model_results["ga"]["mean_auc"]
            ga_std = model_results["ga"]["std_auc"]
            ga_features = model_results["ga"]["common_features"]

            f.write(f"Algoritmo Genético:\n")
            f.write(f"  AUC-ROC: {ga_auc:.4f} ± {ga_std:.4f}\n")
            f.write(f"  Features: {', '.join(ga_features)}\n\n")

            # Determinar o melhor método
            best_auc = max(all_features_auc, anova_auc, ga_auc)

            if best_auc == all_features_auc:
                best_method = "Todas as Features"
                best_features = all_features_list
            elif best_auc == anova_auc:
                best_method = f"ANOVA (k={best_k})"
                best_features = anova_features
            else:
                best_method = "Algoritmo Genético"
                best_features = ga_features

            f.write(f"MELHOR MÉTODO: {best_method}\n")
            f.write(f"  AUC-ROC: {best_auc:.4f}\n")
            f.write(f"  Features: {', '.join(best_features)}\n\n")

            f.write("=" * 80 + "\n\n")

    print(f"Informações das melhores features salvas em {filename}")

## Visualização

In [ ]:
def create_comparison_plots(results):
    """
    Cria box plots comparando os diferentes métodos de seleção de features para cada modelo

    Args:
        results: Dicionário com resultados por modelo e método de seleção
    """
    # Criar diretório para salvar os gráficos
    os.makedirs("comparison_plots", exist_ok=True)

    # Dicionário para armazenar todos os dados para um gráfico comparativo final
    all_comparison_data = []

    # Para cada modelo
    for model_name, model_results in results.items():
        # Dados para o box plot
        comparison_data = {"Método": [], "AUC-ROC": []}

        # Adicionar dados de todos os folds para cada método

        # Todas as features
        method_name = "Todas as Features"
        fold_aucs = model_results["all_features"]["fold_aucs"]
        for auc in fold_aucs:
            comparison_data["Método"].append(method_name)
            comparison_data["AUC-ROC"].append(auc)
            all_comparison_data.append(
                {"Modelo": model_name, "Método": method_name, "AUC-ROC": auc}
            )

        # ANOVA
        method_name = f"ANOVA (k={model_results['anova']['best_k']})"
        fold_aucs = model_results["anova"]["best_results"]["fold_aucs"]
        for auc in fold_aucs:
            comparison_data["Método"].append(method_name)
            comparison_data["AUC-ROC"].append(auc)
            all_comparison_data.append(
                {"Modelo": model_name, "Método": "ANOVA", "AUC-ROC": auc}
            )

        # Algoritmo Genético
        method_name = "Algoritmo Genético"
        fold_aucs = [result["auc"] for result in model_results["ga"]["fold_results"]]
        for auc in fold_aucs:
            comparison_data["Método"].append(method_name)
            comparison_data["AUC-ROC"].append(auc)
            all_comparison_data.append(
                {"Modelo": model_name, "Método": method_name, "AUC-ROC": auc}
            )

        # Criar DataFrame
        df_comparison = pd.DataFrame(comparison_data)

        # Criar box plot
        plt.figure(figsize=(10, 6))
        ax = sns.boxplot(x="Método", y="AUC-ROC", data=df_comparison, palette="Set2")

        # Adicionar pontos individuais
        sns.stripplot(
            x="Método",
            y="AUC-ROC",
            data=df_comparison,
            size=4,
            color="black",
            alpha=0.5,
        )

        # Ajustar elementos visuais
        plt.title(
            f"Comparação de Métodos de Seleção de Features para {model_name}",
            fontsize=14,
        )
        plt.ylabel("AUC-ROC", fontsize=12)
        plt.xlabel("Método de Seleção", fontsize=12)
        plt.grid(axis="y", linestyle="--", alpha=0.7)
        plt.ylim(0.5, 1.0)

        # Adicionar médias de cada método
        for i, method in enumerate(df_comparison["Método"].unique()):
            mean_auc = df_comparison[df_comparison["Método"] == method][
                "AUC-ROC"
            ].mean()
            ax.text(i, 0.52, f"Média: {mean_auc:.4f}", ha="center", fontsize=10)

        plt.tight_layout()
        plt.savefig(
            f'comparison_plots/{model_name.replace(" ", "_")}_comparison.png', dpi=300
        )
        plt.close()

    # Criar DataFrame com todos os dados para comparação final
    df_all = pd.DataFrame(all_comparison_data)

    # Criar um heatmap para comparar todos os modelos e métodos
    plt.figure(figsize=(14, 10))

    # Calcular médias por modelo e método
    pivot_table = df_all.pivot_table(
        index="Modelo", columns="Método", values="AUC-ROC", aggfunc="mean"
    )

    # Criar heatmap
    sns.heatmap(
        pivot_table,
        annot=True,
        fmt=".4f",
        cmap="YlGnBu",
        vmin=0.5,
        vmax=1.0,
        linewidths=0.5,
        cbar_kws={"label": "AUC-ROC médio"},
    )
    plt.title(
        "Comparação de AUC-ROC por Modelo e Método de Seleção de Features", fontsize=16
    )
    plt.ylabel("Modelo", fontsize=14)
    plt.xlabel("Método de Seleção de Features", fontsize=14)
    plt.tight_layout()
    plt.savefig("comparison_plots/all_models_comparison_heatmap.png", dpi=300)
    plt.close()

    # Criar um gráfico de barras para comparar o melhor método por modelo
    best_methods = []
    for model_name, model_results in results.items():
        # Obter AUC para cada método
        all_features_auc = model_results["all_features"]["mean_auc"]
        anova_auc = model_results["anova"]["best_results"]["mean_auc"]
        ga_auc = model_results["ga"]["mean_auc"]

        # Determinar o melhor método
        methods = ["Todas as Features", "ANOVA", "Algoritmo Genético"]
        aucs = [all_features_auc, anova_auc, ga_auc]
        best_idx = np.argmax(aucs)
        best_methods.append(
            {
                "Modelo": model_name,
                "Melhor Método": methods[best_idx],
                "AUC-ROC": aucs[best_idx],
            }
        )

    # Criar DataFrame
    df_best = pd.DataFrame(best_methods)

    # Criar gráfico de barras
    plt.figure(figsize=(14, 8))
    bars = plt.bar(df_best["Modelo"], df_best["AUC-ROC"], color="skyblue")

    # Adicionar texto para cada barra indicando o melhor método
    for i, (bar, method) in enumerate(zip(bars, df_best["Melhor Método"])):
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            height + 0.01,
            method,
            ha="center",
            va="bottom",
            rotation=0,
            fontsize=10,
        )

    plt.title("Melhor Método de Seleção de Features por Modelo", fontsize=16)
    plt.ylabel("AUC-ROC", fontsize=14)
    plt.xlabel("Modelo", fontsize=14)
    plt.ylim(0.5, 1.0)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig("comparison_plots/best_methods_by_model.png", dpi=300)
    plt.close()

    print("Gráficos de comparação gerados e salvos na pasta 'comparison_plots'")


def create_summary_table(results):
    """
    Cria uma tabela resumo comparando os diferentes métodos de seleção de features

    Args:
        results: Dicionário com resultados por modelo e método de seleção
    """
    # Dados para a tabela
    table_data = []

    for model_name, model_results in results.items():
        # Obter dados de cada método
        all_features_auc = model_results["all_features"]["mean_auc"]
        all_features_std = model_results["all_features"]["std_auc"]
        all_features_count = len(model_results["all_features"]["features"])

        best_k = model_results["anova"]["best_k"]
        anova_auc = model_results["anova"]["best_results"]["mean_auc"]
        anova_std = model_results["anova"]["best_results"]["std_auc"]
        anova_count = len(model_results["anova"]["best_results"]["features"])

        ga_auc = model_results["ga"]["mean_auc"]
        ga_std = model_results["ga"]["std_auc"]
        ga_count = len(model_results["ga"]["common_features"])

        # Determinar o melhor método
        methods = ["Todas as Features", f"ANOVA (k={best_k})", "Algoritmo Genético"]
        aucs = [all_features_auc, anova_auc, ga_auc]
        best_idx = np.argmax(aucs)
        best_method = methods[best_idx]

        # Adicionar à tabela
        table_data.append(
            {
                "Modelo": model_name,
                "Todas Features AUC": f"{all_features_auc:.4f} ± {all_features_std:.4f}",
                "Todas Features #": all_features_count,
                "ANOVA AUC": f"{anova_auc:.4f} ± {anova_std:.4f}",
                "ANOVA #": anova_count,
                "GA AUC": f"{ga_auc:.4f} ± {ga_std:.4f}",
                "GA #": ga_count,
                "Melhor Método": best_method,
            }
        )

    # Criar DataFrame
    df_summary = pd.DataFrame(table_data)

    # Salvar como CSV
    df_summary.to_csv("comparison_summary.csv", index=False)

    # Exibir tabela
    print(df_summary)
    print("\nTabela resumo salva em 'comparison_summary.csv'")

In [ ]:
# DEBUG
# for i,data in enumerate(smote_folds_data):
#     print(f"Fold {i+1}:")
#     print(type(data[0]), type(data[1]), type(data[2]), type(data[3]))
#     print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)
#     print(data[0].info())
#     print(data[1].info())
#     print(set(data[2]))
#     print(set(data[3]))
#     print("\n")

## Execução

In [ ]:
def search_best_features():
    try:
        start_time = time.time()
        print("===== INICIANDO O PROCESSO DE SELEÇÃO DE FEATURES =====")
        # Executar a avaliação dos modelos
        print("\nAvaliando modelos com diferentes métodos de seleção de features...")
        results_smote = evaluate_all_models_with_all_feature_selection_methods(
            smote_folds_data
        )
        results_original = evaluate_all_models_with_all_feature_selection_methods(
            original_folds_data
        )

        # Salvar as melhores features em um arquivo texto
        print("\nSalvando as melhores features para cada algoritmo...")
        save_best_features_to_txt(results_smote, "best_features_smote.txt")
        save_best_features_to_txt(results_original, "best_features_original.txt")

        # Exibir tempo de execução
        end_time = time.time()
        total_time = end_time - start_time
        hours, remainder = divmod(total_time, 3600)
        minutes, seconds = divmod(remainder, 60)

        print("\n===== PROCESSO CONCLUÍDO =====")
        print(f"Tempo total de execução: {int(hours)}h {int(minutes)}m {int(seconds)}s")
        return results_smote, results_original
    except Exception as e:
        print(f"\nErro durante a execução: {e}")
        import traceback

        traceback.print_exc()

In [ ]:
results_original, results_smote = search_best_features()

In [ ]:
print("===== RESULTADOS com treinamento sem SMOTE=====")
print(results_original)

In [ ]:
print("===== RESULTADOS com treinamento com SMOTE=====")
print(results_smote)

# 4. Hyperparameter Tunning

Nesta seção utilizaremos Optuna para encontrar os melhores hiperparametros para os modelos abaixo utilizando Optune:



1. Decision Trees
2. Gradient Tree Boosting
3. k-Nearest Neighbors
4. LightGBM
5. Multinomial Logistic Regression
6. Naive Bayes
7. Neural Networks
8. Random Forests
9. Support Vector Machines
10. XGBoost
11. CatBoost



## Função para uso do melhor fold para cada Modelo

In [ ]:
def create_optimized_model_folds(original_folds_data, smote_folds_data):
    """
    Cria folds otimizados para cada modelo usando as melhores features e dataset (original ou SMOTE)

    Args:
        original_folds_data: Lista de tuplas (X_train, X_test, y_train, y_test) do dataset original
        smote_folds_data: Lista de tuplas (X_train, X_test, y_train, y_test) do dataset com SMOTE

    Returns:
        Dict: Dicionário onde cada chave é um modelo e o valor é um dicionário com seus folds otimizados
    """
    # Informações dos melhores modelos a partir da análise
    best_models = {
        "Decision Trees": {
            "dataset": "original",
            "method": "Algoritmo Genético",
            "auc": 0.6682,
            "features": [
                "Stress_Keeps_Patient_from_Sleeping",
                "Dental_Health",
                "Medication_Keeps_Patient_from_Sleeping",
                "Race",
                "Physical_Health",
                "Prescription_Sleep_Medication",
                "Employment",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
                "Trouble_Sleeping",
                "Age",
                "Gender",
                "Uknown_Keeps_Patient_from_Sleeping",
                "Mental_Health",
                "Pain_Keeps_Patient_from_Sleeping",
            ],
        },
        "Gradient Tree Boosting": {
            "dataset": "original",
            "method": "Algoritmo Genético",
            "auc": 0.6588,
            "features": [
                "Race",
                "Medication_Keeps_Patient_from_Sleeping",
                "Uknown_Keeps_Patient_from_Sleeping",
                "Physical_Health",
                "Dental_Health",
                "Employment",
                "Stress_Keeps_Patient_from_Sleeping",
                "Pain_Keeps_Patient_from_Sleeping",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
                "Age",
                "Mental_Health",
                "Gender",
                "Prescription_Sleep_Medication",
                "Trouble_Sleeping",
            ],
        },
        "k-Nearest Neighbors": {
            "dataset": "original",
            "method": "Algoritmo Genético",
            "auc": 0.6488,
            "features": [
                "Physical_Health",
                "Prescription_Sleep_Medication",
                "Stress_Keeps_Patient_from_Sleeping",
                "Age",
                "Mental_Health",
                "Race",
                "Medication_Keeps_Patient_from_Sleeping",
                "Uknown_Keeps_Patient_from_Sleeping",
                "Pain_Keeps_Patient_from_Sleeping",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
                "Employment",
                "Gender",
                "Dental_Health",
                "Trouble_Sleeping",
            ],
        },
        "LightGBM": {
            "dataset": "smote",
            "method": "Algoritmo Genético",
            "auc": 0.6467,
            "features": [
                "Prescription_Sleep_Medication",
                "Race",
                "Age",
                "Employment",
                "Pain_Keeps_Patient_from_Sleeping",
                "Dental_Health",
                "Medication_Keeps_Patient_from_Sleeping",
                "Stress_Keeps_Patient_from_Sleeping",
                "Uknown_Keeps_Patient_from_Sleeping",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
                "Physical_Health",
                "Mental_Health",
                "Gender",
                "Trouble_Sleeping",
            ],
        },
        "Multinomial Logistic Regression": {
            "dataset": "original",
            "method": "Algoritmo Genético",
            "auc": 0.6549,
            "features": [
                "Dental_Health",
                "Stress_Keeps_Patient_from_Sleeping",
                "Medication_Keeps_Patient_from_Sleeping",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
                "Prescription_Sleep_Medication",
                "Race",
                "Physical_Health",
                "Age",
                "Mental_Health",
                "Pain_Keeps_Patient_from_Sleeping",
                "Trouble_Sleeping",
                "Gender",
                "Uknown_Keeps_Patient_from_Sleeping",
                "Employment",
            ],
        },
        "Naive Bayes": {
            "dataset": "original",
            "method": "Algoritmo Genético",
            "auc": 0.6579,
            "features": [
                "Prescription_Sleep_Medication",
                "Medication_Keeps_Patient_from_Sleeping",
                "Dental_Health",
                "Age",
                "Trouble_Sleeping",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
                "Employment",
                "Gender",
                "Physical_Health",
                "Mental_Health",
                "Stress_Keeps_Patient_from_Sleeping",
                "Race",
                "Pain_Keeps_Patient_from_Sleeping",
                "Uknown_Keeps_Patient_from_Sleeping",
            ],
        },
        "Random Forests": {
            "dataset": "original",
            "method": "Algoritmo Genético",
            "auc": 0.6604,
            "features": [
                "Dental_Health",
                "Medication_Keeps_Patient_from_Sleeping",
                "Physical_Health",
                "Race",
                "Prescription_Sleep_Medication",
                "Employment",
                "Pain_Keeps_Patient_from_Sleeping",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
                "Stress_Keeps_Patient_from_Sleeping",
                "Age",
                "Uknown_Keeps_Patient_from_Sleeping",
                "Mental_Health",
                "Trouble_Sleeping",
                "Gender",
            ],
        },
        "Support Vector Machines": {
            "dataset": "smote",
            "method": "Algoritmo Genético",
            "auc": 0.6499,
            "features": [
                "Physical_Health",
                "Employment",
                "Pain_Keeps_Patient_from_Sleeping",
                "Prescription_Sleep_Medication",
                "Dental_Health",
                "Race",
                "Stress_Keeps_Patient_from_Sleeping",
                "Medication_Keeps_Patient_from_Sleeping",
                "Age",
                "Gender",
                "Uknown_Keeps_Patient_from_Sleeping",
                "Trouble_Sleeping",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
                "Mental_Health",
            ],
        },
        "XGBoost": {
            "dataset": "smote",
            "method": "Algoritmo Genético",
            "auc": 0.6559,
            "features": [
                "Dental_Health",
                "Medication_Keeps_Patient_from_Sleeping",
                "Race",
                "Prescription_Sleep_Medication",
                "Physical_Health",
                "Employment",
                "Pain_Keeps_Patient_from_Sleeping",
                "Trouble_Sleeping",
                "Uknown_Keeps_Patient_from_Sleeping",
                "Gender",
                "Age",
                "Stress_Keeps_Patient_from_Sleeping",
                "Mental_Health",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
            ],
        },
        "CatBoost": {
            "dataset": "smote",
            "method": "Algoritmo Genético",
            "auc": 0.6465,
            "features": [
                "Prescription_Sleep_Medication",
                "Physical_Health",
                "Age",
                "Employment",
                "Stress_Keeps_Patient_from_Sleeping",
                "Medication_Keeps_Patient_from_Sleeping",
                "Uknown_Keeps_Patient_from_Sleeping",
                "Race",
                "Bathroom_Needs_Keeps_Patient_from_Sleeping",
                "Pain_Keeps_Patient_from_Sleeping",
                "Trouble_Sleeping",
                "Mental_Health",
                "Gender",
                "Dental_Health",
            ],
        },
    }

    # Combinar os folds em um dicionário
    folds_data = {"original": original_folds_data, "smote": smote_folds_data}

    # Criar dicionário para armazenar os folds otimizados por modelo
    optimized_folds = {}

    # Para cada modelo, criar seus folds otimizados
    for model_name, model_info in best_models.items():
        # Determinar qual conjunto de folds usar
        dataset_type = model_info["dataset"]
        selected_folds = folds_data[dataset_type]

        # Features selecionadas para este modelo
        features = model_info["features"]

        # Criar folds otimizados
        model_folds = []

        for i, (X_train, X_test, y_train, y_test) in enumerate(selected_folds):
            # Filtrar apenas as features selecionadas
            X_train_filtered = X_train[features].copy()
            X_test_filtered = X_test[features].copy()

            # Verificar as distribuições de classes
            if (
                i == 0
            ):  # Mostrar apenas para o primeiro fold para não sobrecarregar a saída
                print(f"\nModelo: {model_name}")
                print(f"Dataset: {dataset_type}")
                print(f"Features selecionadas ({len(features)}): {', '.join(features)}")
                print(f"Distribuição de classes em y_train: {Counter(y_train)}")
                print(f"Distribuição de classes em y_test: {Counter(y_test)}")
                print(f"Dimensões de X_train: {X_train_filtered.shape}")
                print(f"Dimensões de X_test: {X_test_filtered.shape}")

            # Adicionar o fold filtrado
            model_folds.append((X_train_filtered, X_test_filtered, y_train, y_test))

        # Armazenar informações completas para este modelo
        optimized_folds[model_name] = {
            "folds": model_folds,
            "features": features,
            "dataset_type": dataset_type,
            "auc": model_info["auc"],
            "method": model_info["method"],
        }

    # Resumo das configurações ótimas por modelo
    print("\n=== RESUMO DAS CONFIGURAÇÕES ÓTIMAS POR MODELO ===")
    for model_name, model_data in optimized_folds.items():
        print(f"{model_name}:")
        print(f"  Dataset: {model_data['dataset_type']}")
        print(f"  Método: {model_data['method']}")
        print(f"  AUC: {model_data['auc']:.4f}")
        print(f"  Número de features: {len(model_data['features'])}")

    return optimized_folds

## Funções Objetivo para busca de hiperparâmetro

In [ ]:
# Funções objetivo adaptadas para usar folds otimizados (sem mapeamento de classes)
def objective_dt(trial, model_folds):
    """Função objetivo para Decision Trees usando folds otimizados"""
    param = {
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "random_state": 42,
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        model = DecisionTreeClassifier(**param)
        model.fit(X_train, y_train)

        # Calcular AUC
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
            auc_scores.append(auc)

    return np.mean(auc_scores)


def objective_gb(trial, model_folds):
    """Função objetivo para Gradient Boosting usando folds otimizados"""
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "random_state": 42,
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        model = GradientBoostingClassifier(**param)
        model.fit(X_train, y_train)

        # Calcular AUC
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
            auc_scores.append(auc)

    return np.mean(auc_scores)


def objective_knn(trial, model_folds):
    """Função objetivo para K-Nearest Neighbors usando folds otimizados"""
    param = {
        "n_neighbors": trial.suggest_int("n_neighbors", 1, 30),
        "weights": trial.suggest_categorical("weights", ["uniform", "distance"]),
        "algorithm": trial.suggest_categorical(
            "algorithm", ["auto", "ball_tree", "kd_tree", "brute"]
        ),
        "p": trial.suggest_int("p", 1, 2),  # p=1: manhattan, p=2: euclidean
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        model = KNeighborsClassifier(**param)
        model.fit(X_train, y_train)

        # Calcular AUC
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
            auc_scores.append(auc)

    return np.mean(auc_scores)


def objective_lgbm(trial, model_folds):
    """Função objetivo para LightGBM usando folds otimizados"""
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state": 42,
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        model = LGBMClassifier(**param)
        model.fit(X_train, y_train)

        # Calcular AUC
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
            auc_scores.append(auc)

    return np.mean(auc_scores)


def objective_lr(trial, model_folds):
    """Função objetivo para Multinomial Logistic Regression usando folds otimizados"""
    param = {
        "C": trial.suggest_float("C", 1e-4, 1e2, log=True),
        "solver": trial.suggest_categorical(
            "solver", ["newton-cg", "lbfgs", "sag", "saga"]
        ),
        "max_iter": 2000,  # fixo para garantir convergência
        "multi_class": "multinomial",
        "random_state": 42,
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        model = LogisticRegression(**param)
        model.fit(X_train, y_train)

        # Calcular AUC
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
            auc_scores.append(auc)

    return np.mean(auc_scores)


def objective_nb(trial, model_folds):
    """Função objetivo para Naive Bayes usando folds otimizados"""
    param = {
        "var_smoothing": trial.suggest_float("var_smoothing", 1e-12, 1e-2, log=True)
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        model = GaussianNB(**param)
        model.fit(X_train, y_train)

        # Calcular AUC
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
            auc_scores.append(auc)

    return np.mean(auc_scores)


def objective_rf(trial, model_folds):
    """Função objetivo para Random Forests usando folds otimizados"""
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "random_state": 42,
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        model = RandomForestClassifier(**param)
        model.fit(X_train, y_train)

        # Calcular AUC
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
            auc_scores.append(auc)

    return np.mean(auc_scores)


def objective_svm(trial, model_folds):
    """Função objetivo para Support Vector Machines usando folds otimizados"""
    param = {
        "C": trial.suggest_float("C", 1e-4, 1e2, log=True),
        "kernel": trial.suggest_categorical(
            "kernel", ["linear", "poly", "rbf", "sigmoid"]
        ),
        "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
        "probability": True,  # Necessário para calcular AUC
        "random_state": 42,
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        model = SVC(**param)
        model.fit(X_train, y_train)

        # Calcular AUC
        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
            auc_scores.append(auc)

    return np.mean(auc_scores)


def objective_xgb(trial, model_folds):
    """Função objetivo para XGBoost usando folds otimizados"""
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state": 42,
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        try:
            model = XGBClassifier(**param)
            model.fit(X_train, y_train)

            # Calcular AUC
            if hasattr(model, "predict_proba"):
                y_prob = model.predict_proba(X_test)
                auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
                auc_scores.append(auc)
        except Exception as e:
            print(f"Erro ao treinar XGBoost: {e}")

    return (
        np.mean(auc_scores) if auc_scores else 0.0
    )  # Retornar 0 se todas as tentativas falharem


def objective_catboost(trial, model_folds):
    """Função objetivo para CatBoost usando folds otimizados"""
    param = {
        "iterations": trial.suggest_int("iterations", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 10.0, log=True),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),
        "grow_policy": trial.suggest_categorical(
            "grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]
        ),
        "random_seed": 42,
        "verbose": False,
    }

    # Avaliação usando os folds otimizados
    auc_scores = []
    for X_train, X_test, y_train, y_test in model_folds:
        try:
            model = CatBoostClassifier(**param)
            model.fit(X_train, y_train)

            # Calcular AUC
            if hasattr(model, "predict_proba"):
                y_prob = model.predict_proba(X_test)
                auc = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
                auc_scores.append(auc)
        except Exception as e:
            print(f"Erro ao treinar CatBoost: {e}")

    return (
        np.mean(auc_scores) if auc_scores else 0.0
    )  # Retornar 0 se todas as tentativas falharem

## Funções Auxiliares

In [ ]:
def optimize_models_with_optimized_folds(optimized_folds, n_trials=50, timeout=None):
    """
    Otimiza os hiperparâmetros para todos os modelos usando os folds otimizados

    Args:
        optimized_folds: Dicionário com folds otimizados por modelo
        n_trials: Número de tentativas para cada modelo
        timeout: Tempo máximo em segundos para cada modelo (None = sem limite)

    Returns:
        dict: Dicionário com os melhores parâmetros para cada modelo
    """
    # Definir os modelos e suas respectivas funções objetivo
    objective_funcs = {
        "Decision Trees": objective_dt,
        "Gradient Tree Boosting": objective_gb,
        "k-Nearest Neighbors": objective_knn,
        "LightGBM": objective_lgbm,
        "Multinomial Logistic Regression": objective_lr,
        "Naive Bayes": objective_nb,
        "Random Forests": objective_rf,
        "Support Vector Machines": objective_svm,
        "XGBoost": objective_xgb,
        "CatBoost": objective_catboost,
    }

    # Verificar se os nomes dos modelos coincidem com os das funções objetivo
    model_names = set(optimized_folds.keys())
    objective_names = set(objective_funcs.keys())
    missing_models = model_names - objective_names

    if missing_models:
        print(f"Aviso: Modelos sem função objetivo correspondente: {missing_models}")

    # Dicionários para armazenar resultados
    best_params = {}
    best_scores = {}
    optimization_times = {}

    # Para cada modelo, criar e otimizar um estudo
    for model_name, model_data in tqdm(
        optimized_folds.items(), desc="Otimizando modelos"
    ):
        if model_name not in objective_funcs:
            print(f"Pulando {model_name} - função objetivo não disponível")
            continue

        print(
            f"\nOtimizando hiperparâmetros para {model_name} - dataset {model_data['dataset_type']}..."
        )
        start_time = time.time()

        # Obter os folds otimizados para este modelo
        model_folds = model_data["folds"]

        # Criar estudo Optuna
        study = optuna.create_study(
            direction="maximize", study_name=f"{model_name} Optimization"
        )

        # Criar closure para a função objetivo com os folds otimizados
        objective_func = objective_funcs[model_name]
        obj_func = lambda trial: objective_func(trial, model_folds)

        # Otimizar com limite de tempo
        try:
            study.optimize(obj_func, n_trials=n_trials, timeout=timeout)

            # Armazenar os melhores parâmetros e score
            best_params[model_name] = study.best_params
            best_scores[model_name] = study.best_value

            end_time = time.time()
            optimization_times[model_name] = end_time - start_time

            print(f"Melhor score para {model_name}: {study.best_value:.4f}")
            print(f"Melhores parâmetros para {model_name}: {study.best_params}")
            print(f"Tempo de otimização: {optimization_times[model_name]:.2f} segundos")

            # Salvar o estudo
            os.makedirs("optuna_studies", exist_ok=True)
            joblib.dump(
                study,
                f"optuna_studies/{model_name.replace(' ', '_').lower()}_study.pkl",
            )

            # Visualizações
            try:
                os.makedirs("optuna_plots", exist_ok=True)
                # Histórico de otimização
                fig = plot_optimization_history(study)
                fig.write_image(
                    f"optuna_plots/{model_name.replace(' ', '_').lower()}_optimization_history.png"
                )

                # Importância dos parâmetros
                fig = plot_param_importances(study)
                fig.write_image(
                    f"optuna_plots/{model_name.replace(' ', '_').lower()}_param_importances.png"
                )
            except Exception as e:
                print(f"Aviso: Erro ao criar visualizações para {model_name}: {e}")

        except Exception as e:
            print(f"Erro durante a otimização de {model_name}: {e}")
            best_params[model_name] = None
            best_scores[model_name] = float("nan")
            optimization_times[model_name] = float("nan")

    # Exibir resumo de resultados
    print(f"\n=== Resumo dos Resultados de Otimização ===")
    results_df = pd.DataFrame(
        {
            "Modelo": list(best_params.keys()),
            "Melhor AUC": [
                best_scores.get(model, float("nan")) for model in best_params
            ],
            "Tempo (s)": [
                optimization_times.get(model, float("nan")) for model in best_params
            ],
            "Dataset": [
                optimized_folds[model]["dataset_type"] for model in best_params
            ],
        }
    )

    # Ordenar por melhor score
    results_df = results_df.sort_values("Melhor AUC", ascending=False).reset_index(
        drop=True
    )
    print(results_df)

    # Salvar hiperparâmetros em um arquivo TXT
    save_hyperparameters_to_txt(best_params, best_scores, optimized_folds)

    # Criar gráfico de barras para comparar modelos
    plt.figure(figsize=(12, 8))
    colors = plt.cm.viridis(np.linspace(0, 1, len(results_df)))
    ax = sns.barplot(x="Modelo", y="Melhor AUC", data=results_df, palette=colors)
    plt.title("Comparação de AUC para Modelos Otimizados", fontsize=16)
    plt.ylabel("AUC Score", fontsize=14)
    plt.xlabel("Modelo", fontsize=14)
    plt.xticks(rotation=45, ha="right")
    plt.ylim(0.5, 1.0)
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    # Adicionar valores
    for i, v in enumerate(results_df["Melhor AUC"]):
        ax.text(i, v + 0.01, f"{v:.4f}", ha="center")

    plt.tight_layout()
    plt.savefig("model_comparison_optimized.png")
    plt.show()

    return best_params, best_scores


def save_hyperparameters_to_txt(best_params, best_scores, optimized_folds):
    """
    Salva os melhores hiperparâmetros para cada modelo em um arquivo TXT

    Args:
        best_params: Dicionário com os melhores hiperparâmetros para cada modelo
        best_scores: Dicionário com os melhores scores para cada modelo
        optimized_folds: Dicionário com informações sobre os folds otimizados
    """
    with open("best_hyperparameters.txt", "w") as f:
        f.write("MELHORES HIPERPARÂMETROS PARA CADA MODELO\n")
        f.write("=" * 80 + "\n\n")

        # Ordenar modelos por AUC score
        sorted_models = sorted(
            best_params.keys(),
            key=lambda model: best_scores.get(model, 0),
            reverse=True,
        )

        for model_name in sorted_models:
            params = best_params.get(model_name)
            score = best_scores.get(model_name, float("nan"))
            dataset_type = optimized_folds[model_name]["dataset_type"]
            features = optimized_folds[model_name]["features"]

            f.write(f"Modelo: {model_name}\n")
            f.write("-" * 80 + "\n")
            f.write(f"Dataset: {dataset_type}\n")
            f.write(f"Melhor AUC: {score:.4f}\n")
            f.write(f"Número de Features: {len(features)}\n")
            f.write(f"Features: {', '.join(features)}\n\n")
            f.write("Hiperparâmetros:\n")

            if params:
                for param_name, param_value in params.items():
                    f.write(f"  {param_name}: {param_value}\n")
            else:
                f.write(
                    "  Não foi possível otimizar os hiperparâmetros para este modelo.\n"
                )

            f.write("\n" + "=" * 80 + "\n\n")

    print(f"Melhores hiperparâmetros salvos em 'best_hyperparameters.txt'")


def train_and_evaluate_models_with_best_params(optimized_folds, best_params):
    """
    Treina e avalia modelos com os melhores hiperparâmetros usando os folds otimizados

    Args:
        optimized_folds: Dicionário com folds otimizados por modelo
        best_params: Dicionário com os melhores hiperparâmetros para cada modelo

    Returns:
        dict: Dicionário com resultados detalhados
        DataFrame: DataFrame com resultados resumidos
    """
    # Definir os construtores de modelos
    model_constructors = {
        "Decision Trees": DecisionTreeClassifier,
        "Gradient Tree Boosting": GradientBoostingClassifier,
        "k-Nearest Neighbors": KNeighborsClassifier,
        "LightGBM": LGBMClassifier,
        "Multinomial Logistic Regression": LogisticRegression,
        "Naive Bayes": GaussianNB,
        "Random Forests": RandomForestClassifier,
        "Support Vector Machines": SVC,
        "XGBoost": XGBClassifier,
        "CatBoost": CatBoostClassifier,
    }

    # Adicionar parâmetros fixos para alguns modelos
    fixed_params = {
        "Multinomial Logistic Regression": {
            "max_iter": 2000,
            "multi_class": "multinomial",
        },
        "Support Vector Machines": {"probability": True},
    }

    # Dicionário para armazenar resultados
    results = defaultdict(dict)

    # Treinar e avaliar cada modelo
    print("\n=== Treinamento e Avaliação de Modelos Otimizados ===")

    for model_name, constructor in tqdm(
        model_constructors.items(), desc="Treinando modelos"
    ):
        if model_name not in optimized_folds or best_params.get(model_name) is None:
            print(f"Pulando {model_name} - dados ou parâmetros não disponíveis")
            continue

        print(f"\nTreinando e avaliando {model_name}...")

        # Obter parâmetros otimizados
        params = best_params[model_name].copy()

        # Adicionar parâmetros fixos, se necessário
        if model_name in fixed_params:
            params.update(fixed_params[model_name])

        # Adicionar random_state para reprodutibilidade (se aplicável)
        if model_name not in ["k-Nearest Neighbors", "Naive Bayes"]:
            params["random_state"] = 42

        # Obter os folds otimizados para este modelo
        model_folds = optimized_folds[model_name]["folds"]

        # Métricas por fold
        fold_metrics = []

        # Avaliar em cada fold
        for fold_idx, (X_train, X_test, y_train, y_test) in enumerate(model_folds):
            print(f"  Fold {fold_idx+1}/{len(model_folds)}")

            # Inicializar variáveis para evitar UnboundLocalError
            model = None
            f1_score_val = float("nan")  # Substituição da acurácia pelo F1-score
            auc = float("nan")
            conf_matrix = None
            class_report = None

            # Treinar modelo diretamente sem mapeamento de classes
            try:
                model = constructor(**params)
                model.fit(X_train, y_train)

                # Prever - tratamento especial para o CatBoost
                if model_name == "CatBoost":
                    try:
                        # Tentar obter classes diretamente
                        y_pred = model.predict(X_test)
                        # Garantir que são valores escalares
                        if isinstance(y_pred, np.ndarray) and y_pred.ndim > 1:
                            y_pred = np.argmax(y_pred, axis=1)
                    except Exception as e:
                        print(f"Erro ao prever com CatBoost: {e}")
                        # Usar um método alternativo
                        y_prob = model.predict_proba(X_test)
                        y_pred = np.argmax(y_prob, axis=1)
                else:
                    # Para outros modelos
                    y_pred = model.predict(X_test)

                # Calcular AUC score
                if hasattr(model, "predict_proba"):
                    y_prob = model.predict_proba(X_test)
                    auc = roc_auc_score(
                        y_test, y_prob, multi_class="ovr", average="macro"
                    )
                else:
                    auc = float("nan")

                # Calcular métricas - usando F1-score ponderado em vez de acurácia
                f1_score_val = f1_score(y_test, y_pred, average="weighted")
                conf_matrix = confusion_matrix(y_test, y_pred)
                class_report = classification_report(y_test, y_pred, output_dict=True)

            except Exception as e:
                print(f"  Erro ao treinar/avaliar o modelo: {e}")
                # Variáveis já foram inicializadas com valores padrão

            # Armazenar métricas deste fold
            fold_metrics.append(
                {
                    "fold_idx": fold_idx,
                    "f1_score": f1_score_val,  # Substituição da acurácia pelo F1-score
                    "auc": auc,
                    "model": model,  # Pode ser None se houver falha
                    "conf_matrix": conf_matrix,
                    "class_report": class_report,
                }
            )

            # Exibir métricas formatadas corretamente
            print(
                f"    F1-Score: {f'{f1_score_val:.4f}' if not np.isnan(f1_score_val) else 'N/A'}"
            )
            print(f"    AUC: {f'{auc:.4f}' if not np.isnan(auc) else 'N/A'}")

        # Calcular médias (ignorando NaN)
        valid_f1_scores = [
            m["f1_score"] for m in fold_metrics if not np.isnan(m["f1_score"])
        ]
        valid_aucs = [m["auc"] for m in fold_metrics if not np.isnan(m["auc"])]

        mean_f1_score = np.mean(valid_f1_scores) if valid_f1_scores else float("nan")
        mean_auc = np.mean(valid_aucs) if valid_aucs else float("nan")

        # Determinar o melhor fold com base no AUC (se houver)
        if valid_aucs:
            best_fold_idx = np.argmax(
                [m["auc"] if not np.isnan(m["auc"]) else -np.inf for m in fold_metrics]
            )
            best_model = fold_metrics[best_fold_idx]["model"]
        else:
            best_fold_idx = 0
            best_model = None

        # Armazenar resultados deste modelo
        results[model_name] = {
            "fold_metrics": fold_metrics,
            "mean_f1_score": mean_f1_score,  # Substituição da acurácia pelo F1-score
            "mean_auc": mean_auc,
            "dataset_type": optimized_folds[model_name]["dataset_type"],
            "best_fold_idx": best_fold_idx,
            "best_model": best_model,
        }

        print(
            f"  Média de F1-Score: {f'{mean_f1_score:.4f}' if not np.isnan(mean_f1_score) else 'N/A'}"
        )
        print(
            f"  Média de AUC: {f'{mean_auc:.4f}' if not np.isnan(mean_auc) else 'N/A'}"
        )

    # Criar DataFrame com resultados resumidos
    summary_data = []
    for model_name, model_results in results.items():
        summary_data.append(
            {
                "Modelo": model_name,
                "Dataset": model_results["dataset_type"],
                "F1-Score": model_results[
                    "mean_f1_score"
                ],  # Substituição da acurácia pelo F1-score
                "AUC": model_results["mean_auc"],
            }
        )

    results_df = pd.DataFrame(summary_data)
    results_df = results_df.sort_values("AUC", ascending=False).reset_index(drop=True)

    # Exibir tabela de resultados
    print("\n=== Resultados Finais ===")
    print(results_df)

    # Plotar resultados
    plt.figure(figsize=(14, 10))

    # Subplot para F1-Score
    plt.subplot(2, 1, 1)
    colors = plt.cm.viridis(np.linspace(0, 1, len(results_df)))
    ax1 = sns.barplot(x="Modelo", y="F1-Score", data=results_df, palette=colors)
    plt.title("Comparação de F1-Score para Modelos Otimizados", fontsize=16)
    plt.ylabel("F1-Score", fontsize=14)
    plt.ylim(0.5, 1.0)
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    # Adicionar valores
    for i, v in enumerate(results_df["F1-Score"]):
        if not np.isnan(v):
            ax1.text(i, v + 0.01, f"{v:.4f}", ha="center")

    # Subplot para AUC
    plt.subplot(2, 1, 2)
    ax2 = sns.barplot(x="Modelo", y="AUC", data=results_df, palette=colors)
    plt.title("Comparação de AUC para Modelos Otimizados", fontsize=16)
    plt.ylabel("AUC Score", fontsize=14)
    plt.xlabel("Modelo", fontsize=14)
    plt.ylim(0.5, 1.0)
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    # Adicionar valores
    for i, v in enumerate(results_df["AUC"]):
        if not np.isnan(v):
            ax2.text(i, v + 0.01, f"{v:.4f}", ha="center")

    plt.tight_layout()
    plt.savefig("final_model_evaluation.png")
    plt.show()

    # Salvar o melhor modelo geral
    valid_models = results_df[~results_df["AUC"].isna()]
    if not valid_models.empty:
        best_model_idx = valid_models["AUC"].idxmax()
        best_model_row = results_df.iloc[best_model_idx]
        best_model_name = best_model_row["Modelo"]
        best_dataset_name = best_model_row["Dataset"]

        best_model = results[best_model_name]["best_model"]

        # Salvar o modelo
        if best_model is not None:
            os.makedirs("models", exist_ok=True)
            joblib.dump(
                best_model,
                f"models/best_model_{best_model_name.replace(' ', '_').lower()}.pkl",
            )
            print(f"\nMelhor modelo: {best_model_name} (Dataset: {best_dataset_name})")
            print(f"AUC: {best_model_row['AUC']:.4f}")
            print(
                f"F1-Score: {best_model_row['F1-Score']:.4f}"
            )  # Ajustado para exibir F1-Score
            print(
                f"Modelo salvo como 'models/best_model_{best_model_name.replace(' ', '_').lower()}.pkl'"
            )

            # Se for um modelo baseado em árvore, mostrar a importância das features
            if hasattr(best_model, "feature_importances_"):
                features = optimized_folds[best_model_name]["features"]
                importances = pd.DataFrame(
                    {"Feature": features, "Importance": best_model.feature_importances_}
                ).sort_values("Importance", ascending=False)

                plt.figure(figsize=(10, 6))
                sns.barplot(x="Importance", y="Feature", data=importances)
                plt.title(
                    f"Importância das Features para {best_model_name}", fontsize=14
                )
                plt.tight_layout()
                plt.savefig("best_model_feature_importance.png")
                plt.show()

                print("\nImportância das Features:")
                print(importances)
    else:
        print("\nNenhum modelo válido encontrado.")

    return results, results_df

In [ ]:
def buscar_hiperparametros():
    start_time = time.time()

    print("===== INICIANDO O PROCESSO DE OTIMIZAÇÃO COM FOLDS OTIMIZADOS =====")

    # 1. Criar folds otimizados para cada modelo
    print("\nCriando folds otimizados para cada modelo...")
    optimized_folds = create_optimized_model_folds(
        original_folds_data, smote_folds_data
    )

    # 2. Otimizar hiperparâmetros para cada modelo usando os folds otimizados
    print("\nOtimizando hiperparâmetros para cada modelo...")
    best_params, best_scores = optimize_models_with_optimized_folds(
        optimized_folds,
        n_trials=30,  # Reduzido para exemplo
        timeout=600,  # 10 minutos por modelo
    )

    # 3. Treinar e avaliar modelos com os melhores hiperparâmetros
    print("\nTreinando e avaliando modelos com os melhores hiperparâmetros...")
    results, results_df = train_and_evaluate_models_with_best_params(
        optimized_folds, best_params
    )

    # Exibir tempo de execução
    end_time = time.time()
    total_time = end_time - start_time
    hours, remainder = divmod(total_time, 3600)
    minutes, seconds = divmod(remainder, 60)

    print("\n===== PROCESSO CONCLUÍDO =====")
    print(f"Tempo total de execução: {int(hours)}h {int(minutes)}m {int(seconds)}s")

    return optimized_folds, best_params, results, results_df

## Execução da Busca

In [ ]:
optimized_folds, best_params, results, resultds_df = buscar_hiperparametros()

# 5. Avaliação de Modelos

## 5.1 - Comparativo dos Modelos
Nesta subseção calcularemos treinaremos os modelos com os melhores datasets e os melhores hiperparametros encontrados até aqui e faremos um gráfico comparativo das suas AUC_ROC

In [ ]:
def compare_models_performance():
    """
    Compara o desempenho dos modelos treinados com os melhores hiperparâmetros
    em termos de AUC-ROC e F1-score ponderado.
    """
    print("\n" + "=" * 80)
    print("5.1 Comparativo dos Modelos")
    print("=" * 80)

    # Vamos extrair os modelos e seus resultados
    models_summary = []

    for model_name, model_results in results.items():
        best_fold_idx = model_results["best_fold_idx"]
        fold_metrics = model_results["fold_metrics"][best_fold_idx]

        models_summary.append(
            {
                "Modelo": model_name,
                "AUC": model_results["mean_auc"],
                "F1-Score": model_results[
                    "mean_f1_score"
                ],  # Substituído Acurácia por F1-Score
                "Dataset": model_results["dataset_type"],
            }
        )

    # Criar DataFrame com os resultados resumidos
    summary_df = pd.DataFrame(models_summary)
    summary_df = summary_df.sort_values("AUC", ascending=False).reset_index(drop=True)

    # Exibir resultados
    print("\nResumo de desempenho dos modelos ordenados por AUC:")
    print(summary_df)

    # Criar gráfico de barras para comparação visual
    plt.figure(figsize=(14, 10))

    # Gráfico para AUC
    plt.subplot(2, 1, 1)
    sns.barplot(x="Modelo", y="AUC", data=summary_df, palette="viridis")
    plt.title("Comparação de AUC dos Modelos Otimizados", fontsize=16)
    plt.ylim(0.5, 1.0)
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    # Adicionar valores
    for i, v in enumerate(summary_df["AUC"]):
        plt.text(i, v + 0.01, f"{v:.4f}", ha="center", va="bottom")

    # Gráfico para F1-Score (anteriormente Acurácia)
    plt.subplot(2, 1, 2)
    sns.barplot(x="Modelo", y="F1-Score", data=summary_df, palette="viridis")
    plt.title("Comparação de F1-Score Ponderado dos Modelos Otimizados", fontsize=16)
    plt.ylim(0.5, 1.0)
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    # Adicionar valores
    for i, v in enumerate(summary_df["F1-Score"]):
        plt.text(i, v + 0.01, f"{v:.4f}", ha="center", va="bottom")

    plt.tight_layout()
    plt.savefig("models_performance_comparison.png", dpi=300, bbox_inches="tight")
    plt.show()

    return summary_df

In [ ]:
print("\n" + "=" * 80)
print("SEÇÃO 5: AVALIAÇÃO DE MODELOS")
print("=" * 80)
comparison_df = compare_models_performance()

## 5.2 - Comparativo dos Desempenhos por Decil

In [ ]:
def create_decile_plot(optimized_folds, best_params, results, save_path="decile_plots"):
    """
    Cria gráficos de análise por decil para os modelos treinados.
    """
    print("\n" + "=" * 80)
    print("5.2 Análise de Desempenho por Decil")
    print("=" * 80)

    # Criar diretório para salvar os gráficos
    os.makedirs(save_path, exist_ok=True)

    # Para cada modelo
    for model_name, model_results in results.items():
        print(f"\nAnalisando modelo: {model_name}")

        # Obter o melhor modelo deste tipo
        best_model = model_results.get("best_model")
        if best_model is None:
            print(f"  Modelo {model_name} não tem um modelo válido disponível.")
            continue

        # Obter o melhor fold para análise
        best_fold_idx = model_results.get("best_fold_idx", 0)
        try:
            X_test = optimized_folds[model_name]["folds"][best_fold_idx][1]
            y_test = optimized_folds[model_name]["folds"][best_fold_idx][3]
        except (KeyError, IndexError) as e:
            print(f"  Erro ao acessar dados do fold para {model_name}: {e}")
            continue

        try:
            # SOLUÇÃO: Trabalhar com os índices de linha diretamente
            # Criar um array para rastrear o índice original
            row_indices = np.arange(len(X_test))

            # Obter probabilidades ou decisões do modelo
            if hasattr(best_model, "predict_proba"):
                probs = best_model.predict_proba(X_test)
                if probs.shape[1] > 2:  # Multiclasse
                    probabilities = probs.max(axis=1)
                else:  # Binário
                    probabilities = probs[:, 1]
            else:
                # Se não tem probabilidade, usar decisão
                predictions = best_model.predict(X_test)
                probabilities = predictions

            # Preparar dados para análise
            decile_data = {
                "row_idx": row_indices,  # Para rastrear o índice original
                "probability": probabilities,
                "true_class": y_test.values if hasattr(y_test, "values") else y_test,
            }

            # DataFrame para análise
            df = pd.DataFrame(decile_data)

            # Ordenar por probabilidade
            df = df.sort_values("probability", ascending=False)

            # Atribuir decil com base na posição
            n_samples = len(df)
            df["decile"] = np.floor(np.arange(n_samples) / n_samples * 10).astype(int)

            # DataFrame para armazenar métricas por decil
            decile_metrics = pd.DataFrame(index=range(10))
            decile_metrics["count"] = 0
            decile_metrics["f1_score"] = 0.0

            # Calcular métricas por decil
            for decile in range(10):
                decile_df = df[df["decile"] == decile]
                if len(decile_df) > 0:
                    # Obter previsões para este decil usando os índices originais das linhas
                    # CORREÇÃO: Usar os índices de linha rastreados
                    decile_row_indices = decile_df["row_idx"].values
                    decile_X = X_test.iloc[decile_row_indices]
                    decile_y = decile_df["true_class"].values

                    # Calcular previsões
                    if hasattr(best_model, "predict_proba"):
                        if probs.shape[1] > 2:  # Multiclasse
                            decile_preds = best_model.predict(decile_X)
                        else:  # Binário
                            decile_preds = (decile_df["probability"] > 0.5).astype(int)
                    else:
                        decile_preds = decile_df["probability"]

                    # Calcular F1-score para este decil
                    try:
                        f1 = f1_score(decile_y, decile_preds, average="weighted")
                    except Exception as e:
                        print(f"    Erro ao calcular F1-score para decil {decile}: {e}")
                        f1 = 0.0

                    decile_metrics.loc[decile, "count"] = len(decile_df)
                    decile_metrics.loc[decile, "f1_score"] = f1

            # Calcular proporção por decil
            decile_metrics["proportion"] = (
                decile_metrics["count"] / decile_metrics["count"].sum()
            )

            # Criar gráfico
            fig, ax1 = plt.subplots(figsize=(12, 6))

            # Barras para proporção (representatividade)
            ax1.bar(
                decile_metrics.index,
                decile_metrics["proportion"] * 100,
                color="navy",
                alpha=0.8,
                label="Representatividade (%)",
            )
            ax1.set_xlabel("Decil da população")
            ax1.set_ylabel("Representatividade (%)")
            ax1.set_ylim(
                0, decile_metrics["proportion"].max() * 120
            )  # 20% a mais para espaço

            # Linha para F1-score
            ax2 = ax1.twinx()
            ax2.plot(
                decile_metrics.index,
                decile_metrics["f1_score"] * 100,
                color="red",
                marker="o",
                linewidth=2,
                label="F1-score ponderado (%)",
            )
            ax2.set_ylabel("F1-score ponderado (%)")
            ax2.set_ylim(0, 100)

            # Ajustes finais
            plt.title(f"Análise por Decil - {model_name}", fontsize=14)
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines1 + lines2, labels1 + labels2, loc="lower right")
            plt.xticks(range(10))
            plt.grid(axis="y", alpha=0.3)

            plt.tight_layout()
            plt.savefig(
                f"{save_path}/{model_name.replace(' ', '_')}_decile_plot.png", dpi=300
            )
            plt.close()

            print(
                f"  Gráfico de análise por decil criado com sucesso para {model_name}"
            )

        except Exception as e:
            print(f"  Erro ao criar análise por decil para {model_name}: {e}")
            import traceback

            traceback.print_exc()

    print(f"\nAnálise por decil concluída. Gráficos salvos em '{save_path}'")

In [ ]:
create_decile_plot(optimized_folds, best_params, results)

## 5.3 - Diferenciação dos Modelos
Nesta subseção Usaremos a metodoogia do Demsar para testar quais modelos são diferentes entre si

In [ ]:
def test_statistical_differences(summary_df):
    """
    Aplica os testes de Friedman e Nemenyi conforme recomendado por Demšar (2006)
    para identificar diferenças estatisticamente significativas entre os classificadores.
    """
    print("\n" + "=" * 80)
    print("5.2 Diferenciação dos Modelos (Testes Estatísticos)")
    print("=" * 80)

    # Extrair nomes dos modelos e suas métricas
    model_names = summary_df["Modelo"].tolist()
    auc_values = summary_df["AUC"].tolist()

    # Para o teste de Friedman, precisaríamos das performances em cada fold
    # Vamos construir uma matriz com modelos nas colunas e folds nas linhas

    # Primeiro, determinar o número de folds (assumindo que todos os modelos têm mesmo número)
    n_folds = len(results[model_names[0]]["fold_metrics"])

    # Construir matriz de performace para o teste de Friedman
    performance_matrix = np.zeros((n_folds, len(model_names)))

    for j, model_name in enumerate(model_names):
        for i in range(n_folds):
            performance_matrix[i, j] = results[model_name]["fold_metrics"][i]["auc"]

    # Convertendo para DataFrame para melhor visualização
    performance_df = pd.DataFrame(performance_matrix, columns=model_names)

    print("\nMatriz de Performance (AUC) por fold e modelo:")
    print(performance_df)

    # 1. Teste de Friedman
    # O teste de Friedman verifica se há diferenças significativas entre os modelos

    # Realizar o ranking por linha (fold)
    ranks = np.zeros_like(performance_matrix)
    for i in range(n_folds):
        ranks[i, :] = stats.rankdata(
            -performance_matrix[i, :]
        )  # Negativo para ordenar decrescente

    # Média dos ranks por modelo
    mean_ranks = np.mean(ranks, axis=0)

    # DataFrame para melhor visualização
    rank_df = pd.DataFrame(
        {"Modelo": model_names, "Rank Médio": mean_ranks}
    ).sort_values("Rank Médio")

    print("\nRanking médio dos modelos (menor é melhor):")
    print(rank_df)

    # Estatística do teste de Friedman
    k = len(model_names)  # número de modelos
    n = n_folds  # número de folds/datasets

    # Cálculo do Chi-quadrado de Friedman
    chi2 = 12 * n / (k * (k + 1)) * (np.sum(mean_ranks**2) - k * (k + 1) ** 2 / 4)

    # Correção por Iman e Davenport (1980)
    ff = (n - 1) * chi2 / (n * (k - 1) - chi2)

    # Valores críticos para F com (k-1) e (k-1)(n-1) graus de liberdade
    df1 = k - 1
    df2 = (k - 1) * (n - 1)
    p_value = 1 - stats.f.cdf(ff, df1, df2)

    print(f"\nTeste de Friedman com correção de Iman-Davenport:")
    print(f"F({df1}, {df2}) = {ff:.4f}, p-value = {p_value:.4f}")

    alpha = 0.05
    if p_value < alpha:
        print(
            f"Há diferenças estatisticamente significativas entre os modelos (p < {alpha})."
        )

        # 2. Teste post-hoc de Nemenyi
        # O teste de Nemenyi é usado após rejeitar H0 no teste de Friedman
        # para identificar quais pares específicos de modelos são diferentes

        # Diferença crítica para o teste de Nemenyi
        q_alpha = get_nemenyi_critical_value(k, alpha)
        cd = q_alpha * np.sqrt(k * (k + 1) / (6 * n))

        print(f"\nTeste post-hoc de Nemenyi:")
        print(f"Diferença Crítica (CD) = {cd:.4f}")

        # Matriz de diferenças entre pares de classificadores
        diff_matrix = np.zeros((k, k))
        p_matrix = np.zeros((k, k))

        for i in range(k):
            for j in range(i + 1, k):
                diff = abs(mean_ranks[i] - mean_ranks[j])
                diff_matrix[i, j] = diff
                diff_matrix[j, i] = diff

                # Verificar se a diferença é estatisticamente significativa
                if diff > cd:
                    p_matrix[i, j] = 1  # significativo
                    p_matrix[j, i] = 1  # significativo

        # Converter para DataFrames para visualização
        diff_df = pd.DataFrame(diff_matrix, index=model_names, columns=model_names)
        signif_df = pd.DataFrame(p_matrix, index=model_names, columns=model_names)

        print("\nMatriz de diferenças absolutas entre os ranks médios:")
        print(diff_df)

        print("\nMatriz de significância (1 = diferença significativa):")
        print(signif_df)

        # Retornar dados necessários para visualização
        return {
            "model_names": model_names,
            "mean_ranks": mean_ranks,
            "cd": cd,
            "rank_df": rank_df,
            "has_significant_diff": True,
            "performance_df": performance_df,
        }
    else:
        print(
            f"Não há diferenças estatisticamente significativas entre os modelos (p > {alpha})."
        )
        return {
            "model_names": model_names,
            "mean_ranks": mean_ranks,
            "cd": None,
            "rank_df": rank_df,
            "has_significant_diff": False,
            "performance_df": performance_df,
        }


def get_nemenyi_critical_value(k, alpha=0.05):
    """
    Retorna o valor crítico q_alpha para o teste de Nemenyi baseado no número de classificadores.
    Valores baseados na tabela do artigo de Demšar.
    """
    # Valores críticos para q (baseados no artigo de Demšar para alpha = 0.05)
    q_table = {
        2: 1.960,
        3: 2.343,
        4: 2.569,
        5: 2.728,
        6: 2.850,
        7: 2.949,
        8: 3.031,
        9: 3.102,
        10: 3.164,
    }

    if k in q_table:
        return q_table[k]
    else:
        # Para k > 10, uma aproximação pode ser usada
        # Esta é uma simplificação; valores exatos devem ser consultados na literatura
        return 3.164 + (k - 10) * 0.05

In [ ]:
stat_results = test_statistical_differences(comparison_df)

# 5.3 - Avaliação Final
Dentre os modelos que são diferentes entre si, criaremos um gráfico comparativo com suas AUC_ROC. Caso dois modelos sejam equivalentes, ficará o modelo com melhor performance

In [ ]:
def create_cd_diagram(stat_results):
    """
    Cria um Diagrama CD (Critical Difference) conforme proposto por Demšar,
    que visualiza os ranks médios dos classificadores e conecta aqueles que
    não são estatisticamente diferentes.
    """
    print("\n" + "=" * 80)
    print("5.3 Avaliação Final")
    print("=" * 80)

    model_names = stat_results["model_names"]
    mean_ranks = stat_results["mean_ranks"]
    cd = stat_results["cd"]

    # Ordenar modelos por rank médio
    model_ranks = list(zip(model_names, mean_ranks))
    model_ranks.sort(key=lambda x: x[1])  # ordenar por rank

    # Extrair nomes e ranks ordenados
    sorted_names = [x[0] for x in model_ranks]
    sorted_ranks = [x[1] for x in model_ranks]

    # Criar figura
    plt.figure(figsize=(12, 6))

    # Definir eixo do rank
    ax = plt.gca()
    ax.set_xlim(0.5, len(model_names) + 0.5)
    ax.set_ylim(0, 2)

    # Desenhar linha horizontal
    plt.axhline(y=1, color="black", linestyle="-")

    # Marcar posições dos classificadores
    for i, (name, rank) in enumerate(zip(sorted_names, sorted_ranks)):
        position = i + 1
        plt.plot([position, position], [0.9, 1.1], "k-")
        plt.text(position, 0.7, name, ha="center", va="center", rotation=45)
        plt.text(position, 1.3, f"{rank:.2f}", ha="center", va="center")

    # Se há diferença significativa, adicionar CD e conectar grupos não diferentes
    if stat_results["has_significant_diff"] and cd is not None:
        plt.plot([1, 1 + cd], [0.4, 0.4], "k-")
        plt.plot([1, 1], [0.4, 0.45], "k-")
        plt.plot([1 + cd, 1 + cd], [0.4, 0.45], "k-")
        plt.text(1 + cd / 2, 0.3, f"CD = {cd:.2f}", ha="center", va="center")

        # Conectar classificadores que não são significativamente diferentes
        # Esta é uma implementação simplificada; o algoritmo completo é mais complexo
        for i, model_i in enumerate(sorted_names):
            for j, model_j in enumerate(sorted_names):
                if i < j:  # evitar comparações duplicadas
                    idx_i = model_names.index(model_i)
                    idx_j = model_names.index(model_j)

                    if abs(mean_ranks[idx_i] - mean_ranks[idx_j]) <= cd:
                        plt.plot([i + 1, j + 1], [1.7, 1.7], "k-")

    plt.title("Diagrama de Diferença Crítica", fontsize=16)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig("critical_difference_diagram.png", dpi=300, bbox_inches="tight")
    plt.show()

    # Resumo final dos resultados
    print("\nAvaliação Final dos Modelos:")

    if stat_results["has_significant_diff"]:
        print(f"- Há diferenças estatisticamente significativas entre os modelos.")
        print(f"- Diferença Crítica (CD): {cd:.4f}")

        # Identificar grupos de modelos estatisticamente equivalentes
        groups = []
        checked = set()

        for i, model_i in enumerate(sorted_names):
            if model_i in checked:
                continue

            group = [model_i]
            checked.add(model_i)

            for j, model_j in enumerate(sorted_names):
                if model_j in checked or i == j:
                    continue

                idx_i = model_names.index(model_i)
                idx_j = model_names.index(model_j)

                if abs(mean_ranks[idx_i] - mean_ranks[idx_j]) <= cd:
                    group.append(model_j)
                    checked.add(model_j)

            groups.append(group)

        print("\nGrupos de modelos estatisticamente equivalentes:")
        for i, group in enumerate(groups):
            if len(group) > 1:
                print(f"  Grupo {i+1}: {', '.join(group)}")
            else:
                print(f"  Modelo isolado: {group[0]}")

        # Identificar o melhor modelo ou grupo de modelos
        best_rank = min(sorted_ranks)
        best_models = [
            model
            for model, rank in zip(sorted_names, sorted_ranks)
            if rank == best_rank
        ]

        print(f"\nMelhor(es) modelo(s) com rank médio {best_rank:.2f}:")
        for model in best_models:
            print(f"  - {model}")
    else:
        print("- Não há diferenças estatisticamente significativas entre os modelos.")
        print(
            "- Todos os modelos podem ser considerados equivalentes em termos de desempenho."
        )

    # Conclusões e recomendações finais
    print("\nConclusões e Recomendações:")
    if stat_results["has_significant_diff"]:
        best_model = sorted_names[0]
        print(f"- O modelo {best_model} apresentou o melhor desempenho geral.")
        print(
            f"- Recomenda-se utilizar o {best_model} para este problema, considerando:"
        )
        print(f"  * Melhor rank médio: {sorted_ranks[0]:.2f}")
        print(
            f"  * Diferença estatisticamente significativa em relação a alguns modelos."
        )
    else:
        print(
            f"- Como não há diferenças significativas, recomenda-se considerar outros fatores além do desempenho:"
        )
        print(f"  * Tempo de treinamento e inferência")
        print(f"  * Interpretabilidade do modelo")
        print(f"  * Requisitos computacionais")
        print(f"  * Facilidade de manutenção e atualização")

In [ ]:
create_cd_diagram(stat_results)